# Computing Centralizers of third order operators

In [1]:
import sys
sys.path.insert(0,"..") # salgebra is here

from dalgebra import *
%display latex

In this notebook we show how to use `dalgebra` for computing centralizers (or at least, the conditions to obtain a centralizer) for some operators. This is based on results from 
* [6] G. Wilson, _Algebraic curves and soluton equations_, Geometry Today, 1985, pp. 303-329,

and also in some current and recent reasearch from:
* [3] J.J. Morales-Ruiz, S.L. Rueda and M.A. Zurro, _Factorization of KdV Schrödinger operators using differential subresultants_, Adv. Appl. Math. **120** (2020), 102065
* [4] E. Oreviato, Sl.L. Rueda and M.A. Zurro, _Commuting Ordinary Differential OPerators and the Dixmier Test_, "SIGMA (Symmetry, integrability and Geometry: Methods and Application)" **15** (2019) no. 101, 23 pp.

## The problem

Let us consider a linear differential operator of the form:
$$L = \partial^n  + u_{n-1}\partial^{n-2} + \ldots + u_1\partial + u_0,$$
where the variables $u_*$ are in some differential field $K$ with constant field within $\mathbb{C}$. The centralizers of these operators have quotient fields that can be seen as affine rings over curves. This can be used to described and understand better the centralizers of $L$ and some of its properties.

**Definition** _(almost commuting)_: let $A,B \in K[\partial]$ be of order $n$ and $m$ respectively. We say that $B$ almost commute with $A$ if $\text{ord}([A,B]) \leq n-2$.

By definition, two operators commute when $[A,B] = 0$ and, in general, this commutator have order $n+m-1$. Hence, if the order of the commutator is _so small_ that is smaller than $n-1$, then we consider the operator $B$ _almost commute_ with $A$. 

**_Remark_**: _almost commuting_ is not a symmetric property: if $\text{ord}([A,B]) = n-2$ and $n > m$ then $B$ almost commutes with $A$ but $A$ does not almost commute with $B$.

### Almost commuting operators: their structure

**Lemma** let $A \in K[\partial]$. Then the set of $B$ such that $B$ almost commute with $A$ is a $\mathbb{C}$-vector space.

_Proof:_ this is trivial using the properties of $[\cdot,\cdot]$:
* $[A, B+C] = AB + AC - BA - CA = [A,B] + [A,C]$. Hence $\text{ord}([A,B+C]) \leq \max\{\text{ord}([A,B]), \text{ord}([A,C])\} \leq n-2$.
* If $\partial(c) = 0$, then $[A,cB] = c[A,B]$. Hence, $\text{ord}([A,cB]) = \text{ord}([A,B]) \leq n-2$.

As it usually happens with this type of vector spaces of operators, we can consider them sliced into the orders of the operators. In particular, we have that the sets
$$AC_m(A) = \left\{B \in K[\partial]\ :\ \text{$B$ almost commute with $A$ and $\text{ord}(B) \leq m$}\right\}$$
are subspaces of the space of almost commuting operators.

It was shown in [6] that the vector spaces $AC_m(A)$ has exactly dimension $m$. This, together with the fact that $AC_m(A) \subset AC_{m+1}(A)$, means that there is an operator of order exactly $m$ that almost commute with $A$. This operator can be uniquely defined if provided more properties.

**_Remark_**: if $B$ commutes with $A$, then $B$ almost commute with $A$.

**Theorem [6]** Let $L = \partial^n  + u_{n-2}\partial^{n-2} + \ldots + u_1\partial + u_0$ and assign weights $u_{j}^{(k)} = n-j+k$. Then there is a unique basis $\{P_m\}\subset R\{u_0,\ldots,u_{n-2}\}[\partial]$ such that
* $P_m = \partial_m + p_{m,m-2}\partial^{m-2} + ... + p_{m,1}\partial + p_{m,0}$.
* The coefficients $p_{m,j}$ is homogeneous of weight $m-j$.

### Using `dalgebra` to compute the Wilson basis for $L$ up to order $m$

First, we start by defining the basic system that need to be satisfied. We start by creating the d-variables for $L$ and $P_m$:

In [3]:
from itertools import accumulate
n=3; m=5
names_u = ["u"] if n == 2 else [f"u{i}" for i in range(n-1)]
names_Y = ["Y"] if m == 2 else [f"Y{i}" for i in range(m-1)]
R = DifferentialPolynomialRing(QQ, names_Y + names_u + ["z"]); R
Y, u, z = R.gens()[:m-1], R.gens()[m-1:m+n-2], R.gens()[-1]

We then proceed to compute the commutator between $L$ and $P$:

In [4]:
L = z[n] + sum(z[i]*u[i][0] for i in range(n-1))
P = z[m] + sum(z[i]*Y[i][0] for i in range(m-1))
C = L(z=P) - P(z=L)
system = DifferentialSystem([C.coefficient(z[i]) for i in range(n-1, C.order(z)+1)], variables=Y)
system

System over [Ring of operator polynomials in (Y0, Y1, Y2, Y3, u0, u1, z) over Differential Ring [[Rational Field], (0,)]] with variables [(Y0_*, Y1_*, Y2_*, Y3_*)]:
{
	Y2_1*u1_0 + (-2)*Y2_0*u1_1 + (-3)*Y3_0*u0_1 + (-3)*Y3_0*u1_2 + 3*Y0_1 + 3*Y1_2 + Y2_3 + (-10)*u0_3 + (-5)*u1_4 == 0
	Y3_1*u1_0 + (-3)*Y3_0*u1_1 + 3*Y1_1 + 3*Y2_2 + Y3_3 + (-10)*u0_2 + (-10)*u1_3 == 0
	3*Y2_1 + 3*Y3_2 + (-5)*u0_1 + (-10)*u1_2 == 0
	3*Y3_1 + (-5)*u1_1 == 0
}

If we now impose the Wilson weights and the desired weights for the coefficients $Yj_*$ we can check this system is of homogeneous equations:

In [5]:
weight = R.weight_func(list(range(m,1,-1))+list(range(n,1,-1)) + [0],[1])
all(equation.is_homogeneous(weight) for equation in system.equations())

True

We can then set up ansatz for the unknowns of the corresponding orders and solve the corresponding linear system that arises:

In [6]:
weight = R.weight_func((m-1)*[0]+list(range(n,1,-1))+[0],[1])
ansatz_monoms = [weight.homogeneous_monomials(m-i) for i in range(m-1)]
lengths = [len(mons) for mons in ansatz_monoms]
num_constants = sum(len(mons) for mons in ansatz_monoms)

In [7]:
BwC = PolynomialRing(QQ, [f"c_{j}_{i}" for (j,mons) in enumerate(ansatz_monoms) for i in range(len(mons))])
cs = BwC.gens()
c = [cs[x - y: x] for x, y in zip(accumulate(lengths), lengths)]
RwC = R.change_ring(DifferentialRing(BwC, lambda p : 0))
Y, u, z = RwC.gens()[:m-1], RwC.gens()[m-1:m+n-2], RwC.gens()[-1]
## Creating the ansatzs
Yc = {Y[i] : sum(coeff*RwC(monom) for (coeff,monom) in zip(c[i], ansatz_monoms[i])) for i in range(m-1)}; Yc

{Y0_*: c_0_3*u0_0*u1_0 + c_0_2*u1_1*u1_0 + c_0_1*u0_2 + c_0_0*u1_3,
 Y1_*: c_1_1*u1_0^2 + c_1_0*u0_1 + c_1_2*u1_2,
 Y2_*: c_2_1*u0_0 + c_2_0*u1_1,
 Y3_*: c_3_0*u1_0}

The next step is to plug these ansatz into the equations from the system. Then we set all the coefficients to be zero and we obtain a lienar system in the ansatz variables that will define a almost commuting operator for order $m$:

In [8]:
## Building the linear system
CwC = RwC(C)
equations = sum([CwC.coefficient(z[i])(dic=Yc).coefficients() for i in range(n-1, CwC.order(z)+1)], [])
A = Matrix([[equ.wrapped.coefficient(v) for v in cs] for equ in equations]); b = vector([equ.wrapped.constant_coefficient() for equ in equations])
sols = A.solve_right(b)

## Building output ring
output_ring = DifferentialPolynomialRing(QQ, names_u + ["z"])
output_z = output_ring.gens()[-1]

## Computing the output operator
sols = [[output_ring.base()(el) for el in sols[x - y: x]] for x, y in zip(accumulate(lengths), lengths)]
realY = [sum(coeff*output_ring(monom) for (coeff,monom) in zip(sols[i], ansatz_monoms[i])) for i in range(m-1)]
P = sum(realY[i]*output_z[i] for i in range(m-1)) + z[m]; P
final_C = output_ring(C(dic={Y[i]: realY[i] for i in range(m-1)}))
T = [final_C.coefficient(output_z[i]) for i in range(n-1)]
(P, T)

((-10/9)*u0_0*u1_0*z_0 + (-5/9)*u1_0^2*z_1 + (-10/9)*u0_2*z_0 + (-5/3)*u0_1*z_1 + (-5/3)*u0_0*z_2 + (-10/9)*u1_2*z_1 + (-5/3)*u1_1*z_2 + (-5/3)*u1_0*z_3 + z_5,
 [(-5/9)*u0_1*u1_0^2 + (-10/9)*u0_0*u1_1*u1_0 + (-5/9)*u0_3*u1_0 + 5/3*u0_2*u0_0 + (-5/3)*u0_2*u1_1 + 5/3*u0_1^2 + (-20/9)*u0_1*u1_2 + (-10/9)*u0_0*u1_3 + (-19/9)*u0_5,
  (-5/9)*u1_1*u1_0^2 + 10/3*u0_1*u0_0 + (-5/3)*u0_1*u1_1 + (-5/3)*u0_0*u1_2 + (-5/9)*u1_3*u1_0 + (-5/9)*u1_2*u1_1 + (-10)*u0_4 + (-19/9)*u1_5])

In [2]:
## Putting everything into one method
def quasi_commutting_generic_linear(n,m):
    from itertools import accumulate
    
    ## Building the set up for the problem
    names_u = ["u"] if n == 2 else [f"u{i}" for i in range(n-1)]
    names_Y = ["Y"] if m == 2 else [f"Y{i}" for i in range(m-1)]
    R = DifferentialPolynomialRing(QQ, names_Y + names_u + ["z"]); R
    Y, u, z = R.gens()[:m-1], R.gens()[m-1:m+n-2], R.gens()[-1]
    
    ## Building the operator L, P and their commutator
    L = z[n] + sum(z[i]*u[i][0] for i in range(n-1))
    P = z[m] + sum(z[i]*Y[i][0] for i in range(m-1))
    C = L(z=P) - P(z=L)
    
    ## Building the weight function and the homogeneous monomials for appropriate ansatzs
    weight = R.weight_func((m-1)*[0]+list(range(n,1,-1))+[0],[1])
    ansatz_monoms = [weight.homogeneous_monomials(m-i) for i in range(m-1)]
    lengths = [len(mons) for mons in ansatz_monoms]
    num_constants = sum(len(mons) for mons in ansatz_monoms)
    print(f"Number of ansatz constants: {num_constants}")
    
    ## Creating the ansatzs
    BwC = PolynomialRing(QQ, [f"c_{j}_{i}" for (j,mons) in enumerate(ansatz_monoms) for i in range(len(mons))])
    cs = BwC.gens()
    c = [cs[x - y: x] for x, y in zip(accumulate(lengths), lengths)]
    RwC = R.change_ring(DifferentialRing(BwC, lambda p : 0))
    Y, u, z = RwC.gens()[:m-1], RwC.gens()[m-1:m+n-2], RwC.gens()[-1]
    
    Yc = {Y[i] : sum(coeff*RwC(monom) for (coeff,monom) in zip(c[i], ansatz_monoms[i])) for i in range(m-1)}; Yc
    
    ## Building the linear system
    CwC = RwC(C)
    equations = sum([CwC.coefficient(z[i])(dic=Yc).coefficients() for i in range(n-1, CwC.order(z)+1)], [])
    if num_constants == 1: # univariate polynomials are the base structure
        A = Matrix([[equ.wrapped.lc() for v in cs] for equ in equations])
    else: # multivariate polynomials are the base structure
        A = Matrix([[equ.wrapped.coefficient(v) for v in cs] for equ in equations])
    b = vector([equ.wrapped.constant_coefficient() for equ in equations])
    sols = A.solve_right(-b)

    ## Building output ring
    output_ring = DifferentialPolynomialRing(QQ, names_u + ["z"])
    output_z = output_ring.gens()[-1]

    ## Computing the output operator
    sols = [[output_ring.base()(el) for el in sols[x - y: x]] for x, y in zip(accumulate(lengths), lengths)]
    print(sols)
    realY = [sum(coeff*output_ring(monom) for (coeff,monom) in zip(sols[i], ansatz_monoms[i])) for i in range(m-1)]
    print(output_z)
    realP = sum(realY[i]*output_z[i] for i in range(m-1)) + output_z[m]
    final_C = output_ring(C(dic={Y[i]: realY[i] for i in range(m-1)}))
    T = [final_C.coefficient(output_z[i]) for i in range(n-1)]
    return realP,tuple(T) 

In [3]:
########################################################################################
###
### OLD VERSION OF THE ALGORITHM: solves a differential system to obtain a new element
###
########################################################################################
def quasi_commutting_generic_diff(n, m):
    names_u = ["u"] if n == 2 else [f"u{i}" for i in range(n-1)]
    names_Y = ["Y"] if m == 2 else [f"Y{j}" for j in range(m-1)]
    R = DifferentialPolynomialRing(QQ, names_u + names_Y + ["z"])
    Y, u, z = R.gens()[:m-1], R.gens()[m-1:m+n-2], R.gens()[-1]

    ## Creating the two operators L and P
    L = z[n] + sum(z[i]*u[i][0] for i in range(n-1))
    P = z[m] + sum(z[i]*Y[i][0] for i in range(m-1))

    ## Computing commutator
    commutator = L(z=P) - P(z=L)
    system = DifferentialSystem([commutator.coefficient(z[i]) for i in range(n-1, commutator.order(z)+1)], variables=Y)
    #show(system)
    sols = system.solve_linear()
    
    ## Output of method
    RO_u = DifferentialPolynomialRing(QQ, names_u)
    RO = DifferentialPolynomialRing(RO_u, ["z"])
    P_eval = RO(P(**{k.variable_name() : v for (k,v) in sols.items()}))
    commutator_eval = commutator(**{k.variable_name() : v for (k,v) in sols.items()})
    T = [RO_u(commutator_eval.coefficient(z[i])) for i in range(n-1)]
    
    return P_eval, tuple(T)

def get_sequence(n, bound, method):
    sequence = []
    for i in range(n, bound+1):
        P, T = method(n,i)
        
        print("-------------------------------------------------------------------------------------------------------------------")
        print(f"Computed the almost commutator of order {i}")
        show(f"P_{i} = " + latex(P))

        if all(t == 0 for t in T):
            print("... This commutes with L")
            sequence.append((P, tuple([])))
        else:
            ltx_code = r"\left\{\begin{array}{rcl}"
            ltx_code += r"\\".join(r"f_{" + f"{i//n},{j}" + r"} & = & " + latex(T[j]) for j in range(len(T)))
            ltx_code += r"\end{array}\right."
            sequence.append((P, tuple([t for t in T if t != 0])))
            show(LatexExpr(ltx_code))
    return sequence

## Recovering the kdv sequence

In [111]:
%time seq=get_sequence(2, 10, quasi_commutting_generic_linear)

Number of ansatz constants: 1
[[1]]
Generator for the z's in Ring of operator polynomials in (u, z) over Differential Ring [[Rational Field], (0,)]
-------------------------------------------------------------------------------------------------------------------
Computed the almost commutator of order 2


P_2 = 1 u_{0} z_{0} + 1 z_{2}

... This commutes with L
Number of ansatz constants: 2
[[3/4], [3/2]]
Generator for the z's in Ring of operator polynomials in (u, z) over Differential Ring [[Rational Field], (0,)]
-------------------------------------------------------------------------------------------------------------------
Computed the almost commutator of order 3


P_3 = \frac{3}{4} u_{1} z_{0} + \frac{3}{2} u_{0} z_{1} + 1 z_{3}

\left\{\begin{array}{rcl}f_{1,0} & = & \left(-\frac{3}{2}\right) u_{1} u_{0} + \left(-\frac{1}{4}\right) u_{3}\end{array}\right.

...

P_9 = \frac{315}{64} u_{1} u_{0}^{3} z_{0} + \frac{315}{128} u_{0}^{4} z_{1} + \frac{945}{128} u_{3} u_{0}^{2} z_{0} + \frac{945}{32} u_{2} u_{1} u_{0} z_{0} + \frac{1575}{64} u_{2} u_{0}^{2} z_{1} + \frac{945}{128} u_{1}^{3} z_{0} + \frac{2205}{64} u_{1}^{2} u_{0} z_{1} + \frac{945}{32} u_{1} u_{0}^{2} z_{2} + \frac{105}{16} u_{0}^{3} z_{3} + \frac{567}{128} u_{5} u_{0} z_{0} + \frac{1743}{128} u_{4} u_{1} z_{0} + \frac{1449}{64} u_{4} u_{0} z_{1} + \frac{3045}{128} u_{3} u_{2} z_{0} + \frac{1827}{32} u_{3} u_{1} z_{1} + \frac{1575}{32} u_{3} u_{0} z_{2} + \frac{5061}{128} u_{2}^{2} z_{1} + \frac{1575}{16} u_{2} u_{1} z_{2} + \frac{945}{16} u_{2} u_{0} z_{3} + \frac{1365}{32} u_{1}^{2} z_{3} + \frac{315}{8} u_{1} u_{0} z_{4} + \frac{63}{8} u_{0}^{2} z_{5} + \frac{255}{256} u_{7} z_{0} + \frac{897}{128} u_{6} z_{1} + \frac{1407}{64} u_{5} z_{2} + \frac{1281}{32} u_{4} z_{3} + \frac{735}{16} u_{3} z_{4} + \frac{273}{8} u_{2} z_{5} + \frac{63}{4} u_{1} z_{6} + \frac{9}{2} u_{0} z_{7} + 1 z_{9}

\left\{\begin{array}{rcl}f_{4,0} & = & \left(-\frac{315}{128}\right) u_{1} u_{0}^{4} + \left(-\frac{105}{64}\right) u_{3} u_{0}^{3} + \left(-\frac{315}{32}\right) u_{2} u_{1} u_{0}^{2} + \left(-\frac{315}{64}\right) u_{1}^{3} u_{0} + \left(-\frac{63}{128}\right) u_{5} u_{0}^{2} + \left(-\frac{189}{64}\right) u_{4} u_{1} u_{0} + \left(-\frac{315}{64}\right) u_{3} u_{2} u_{0} + \left(-\frac{483}{128}\right) u_{3} u_{1}^{2} + \left(-\frac{651}{128}\right) u_{2}^{2} u_{1} + \left(-\frac{9}{128}\right) u_{7} u_{0} + \left(-\frac{9}{32}\right) u_{6} u_{1} + \left(-\frac{21}{32}\right) u_{5} u_{2} + \left(-\frac{63}{64}\right) u_{4} u_{3} + \left(-\frac{1}{256}\right) u_{9}\end{array}\right.

Number of ansatz constants: 41
[[24, 10, 1, 35, 50, 1, 18, 38, 52, 10, 5, 20], [20, 92, 30, 40, 160, 160, 40, 8], [202, 70, 29, 5, 139, 80, 100], [60, 120, 240, 62], [86, 80, 110, 10], [80, 60], [50, 10], [20], [5]]
Generator for the z's in Ring of operator polynomials in (u, z) over Differential Ring [[Rational Field], (0,)]
-------------------------------------------------------------------------------------------------------------------
Computed the almost commutator of order 10


P_10 = 1 u_{0}^{5} z_{0} + 10 u_{2} u_{0}^{3} z_{0} + 20 u_{1}^{2} u_{0}^{2} z_{0} + 20 u_{1} u_{0}^{3} z_{1} + 5 u_{0}^{4} z_{2} + 10 u_{4} u_{0}^{2} z_{0} + 50 u_{3} u_{1} u_{0} z_{0} + 40 u_{3} u_{0}^{2} z_{1} + 35 u_{2}^{2} u_{0} z_{0} + 52 u_{2} u_{1}^{2} z_{0} + 160 u_{2} u_{1} u_{0} z_{1} + 70 u_{2} u_{0}^{2} z_{2} + 40 u_{1}^{3} z_{1} + 100 u_{1}^{2} u_{0} z_{2} + 60 u_{1} u_{0}^{2} z_{3} + 10 u_{0}^{3} z_{4} + 5 u_{6} u_{0} z_{0} + 18 u_{5} u_{1} z_{0} + 30 u_{5} u_{0} z_{1} + 38 u_{4} u_{2} z_{0} + 92 u_{4} u_{1} z_{1} + 80 u_{4} u_{0} z_{2} + 24 u_{3}^{2} z_{0} + 160 u_{3} u_{2} z_{1} + 202 u_{3} u_{1} z_{2} + 120 u_{3} u_{0} z_{3} + 139 u_{2}^{2} z_{2} + 240 u_{2} u_{1} z_{3} + 110 u_{2} u_{0} z_{4} + 80 u_{1}^{2} z_{4} + 60 u_{1} u_{0} z_{5} + 10 u_{0}^{2} z_{6} + 1 u_{8} z_{0} + 8 u_{7} z_{1} + 29 u_{6} z_{2} + 62 u_{5} z_{3} + 86 u_{4} z_{4} + 80 u_{3} z_{5} + 50 u_{2} z_{6} + 20 u_{1} z_{7} + 5 u_{0} z_{8} + 1 z_{10}

... This commutes with L
CPU times: user 9.01 s, sys: 79.7 ms, total: 9.09 s
Wall time: 9.06 s


In [98]:
%time seq=get_sequence(2, 10, quasi_commutting_generic_diff)

-------------------------------------------------------------------------------------------------------------------
Computed the almost commutator of order 2


P_2 = 1 u_{0} z_{0} + 1 z_{2}

... This commutes with L
-------------------------------------------------------------------------------------------------------------------
Computed the almost commutator of order 3


P_3 = \frac{3}{4} u_{1} z_{0} + \frac{3}{2} u_{0} z_{1} + 1 z_{3}

\left\{\begin{array}{rcl}f_{1,0} & = & \left(-\frac{3}{2}\right) u_{1} u_{0} + \left(-\frac{1}{4}\right) u_{3}\end{array}\right.

...

P_9 = \frac{315}{64} u_{1} u_{0}^{3} z_{0} + \frac{315}{128} u_{0}^{4} z_{1} + \frac{945}{128} u_{3} u_{0}^{2} z_{0} + \frac{945}{32} u_{2} u_{1} u_{0} z_{0} + \frac{1575}{64} u_{2} u_{0}^{2} z_{1} + \frac{945}{128} u_{1}^{3} z_{0} + \frac{2205}{64} u_{1}^{2} u_{0} z_{1} + \frac{945}{32} u_{1} u_{0}^{2} z_{2} + \frac{105}{16} u_{0}^{3} z_{3} + \frac{567}{128} u_{5} u_{0} z_{0} + \frac{1743}{128} u_{4} u_{1} z_{0} + \frac{1449}{64} u_{4} u_{0} z_{1} + \frac{3045}{128} u_{3} u_{2} z_{0} + \frac{1827}{32} u_{3} u_{1} z_{1} + \frac{1575}{32} u_{3} u_{0} z_{2} + \frac{5061}{128} u_{2}^{2} z_{1} + \frac{1575}{16} u_{2} u_{1} z_{2} + \frac{945}{16} u_{2} u_{0} z_{3} + \frac{1365}{32} u_{1}^{2} z_{3} + \frac{315}{8} u_{1} u_{0} z_{4} + \frac{63}{8} u_{0}^{2} z_{5} + \frac{255}{256} u_{7} z_{0} + \frac{897}{128} u_{6} z_{1} + \frac{1407}{64} u_{5} z_{2} + \frac{1281}{32} u_{4} z_{3} + \frac{735}{16} u_{3} z_{4} + \frac{273}{8} u_{2} z_{5} + \frac{63}{4} u_{1} z_{6} + \frac{9}{2} u_{0} z_{7} + 1 z_{9}

\left\{\begin{array}{rcl}f_{4,0} & = & \left(-\frac{315}{128}\right) u_{1} u_{0}^{4} + \left(-\frac{105}{64}\right) u_{3} u_{0}^{3} + \left(-\frac{315}{32}\right) u_{2} u_{1} u_{0}^{2} + \left(-\frac{315}{64}\right) u_{1}^{3} u_{0} + \left(-\frac{63}{128}\right) u_{5} u_{0}^{2} + \left(-\frac{189}{64}\right) u_{4} u_{1} u_{0} + \left(-\frac{315}{64}\right) u_{3} u_{2} u_{0} + \left(-\frac{483}{128}\right) u_{3} u_{1}^{2} + \left(-\frac{651}{128}\right) u_{2}^{2} u_{1} + \left(-\frac{9}{128}\right) u_{7} u_{0} + \left(-\frac{9}{32}\right) u_{6} u_{1} + \left(-\frac{21}{32}\right) u_{5} u_{2} + \left(-\frac{63}{64}\right) u_{4} u_{3} + \left(-\frac{1}{256}\right) u_{9}\end{array}\right.

-------------------------------------------------------------------------------------------------------------------
Computed the almost commutator of order 10


P_10 = 1 u_{0}^{5} z_{0} + 10 u_{2} u_{0}^{3} z_{0} + 20 u_{1}^{2} u_{0}^{2} z_{0} + 20 u_{1} u_{0}^{3} z_{1} + 5 u_{0}^{4} z_{2} + 10 u_{4} u_{0}^{2} z_{0} + 50 u_{3} u_{1} u_{0} z_{0} + 40 u_{3} u_{0}^{2} z_{1} + 35 u_{2}^{2} u_{0} z_{0} + 52 u_{2} u_{1}^{2} z_{0} + 160 u_{2} u_{1} u_{0} z_{1} + 70 u_{2} u_{0}^{2} z_{2} + 40 u_{1}^{3} z_{1} + 100 u_{1}^{2} u_{0} z_{2} + 60 u_{1} u_{0}^{2} z_{3} + 10 u_{0}^{3} z_{4} + 5 u_{6} u_{0} z_{0} + 18 u_{5} u_{1} z_{0} + 30 u_{5} u_{0} z_{1} + 38 u_{4} u_{2} z_{0} + 92 u_{4} u_{1} z_{1} + 80 u_{4} u_{0} z_{2} + 24 u_{3}^{2} z_{0} + 160 u_{3} u_{2} z_{1} + 202 u_{3} u_{1} z_{2} + 120 u_{3} u_{0} z_{3} + 139 u_{2}^{2} z_{2} + 240 u_{2} u_{1} z_{3} + 110 u_{2} u_{0} z_{4} + 80 u_{1}^{2} z_{4} + 60 u_{1} u_{0} z_{5} + 10 u_{0}^{2} z_{6} + 1 u_{8} z_{0} + 8 u_{7} z_{1} + 29 u_{6} z_{2} + 62 u_{5} z_{3} + 86 u_{4} z_{4} + 80 u_{3} z_{5} + 50 u_{2} z_{6} + 20 u_{1} z_{7} + 5 u_{0} z_{8} + 1 z_{10}

... This commutes with L
CPU times: user 5.87 s, sys: 37.2 ms, total: 5.91 s
Wall time: 5.9 s


## Getting the sequence for operator of order 3

In [112]:
%time seq=get_sequence(3, 10, quasi_commutting_generic_linear)

Number of ansatz constants: 3
[[1, 0], [1]]
Generator for the z's in Ring of operator polynomials in (u0, u1, z) over Differential Ring [[Rational Field], (0,)]
-------------------------------------------------------------------------------------------------------------------
Computed the almost commutator of order 3


P_3 = 1 \mathit{u0}_{0} z_{0} + 1 \mathit{u1}_{0} z_{1} + 1 z_{3}

... This commutes with L
Number of ansatz constants: 6
[[2/9, 2/3, 2/9], [4/3, 2/3], [4/3]]
Generator for the z's in Ring of operator polynomials in (u0, u1, z) over Differential Ring [[Rational Field], (0,)]
-------------------------------------------------------------------------------------------------------------------
Computed the almost commutator of order 4


P_4 = \frac{2}{9} \mathit{u1}_{0}^{2} z_{0} + \frac{2}{3} \mathit{u0}_{1} z_{0} + \frac{4}{3} \mathit{u0}_{0} z_{1} + \frac{2}{9} \mathit{u1}_{2} z_{0} + \frac{2}{3} \mathit{u1}_{1} z_{1} + \frac{4}{3} \mathit{u1}_{0} z_{2} + 1 z_{4}

\left\{\begin{array}{rcl}f_{1,0} & = & \frac{4}{9} \mathit{u1}_{1} \mathit{u1}_{0}^{2} + \left(-\frac{2}{3}\right) \mathit{u0}_{2} \mathit{u1}_{0} + \left(-\frac{4}{3}\right) \mathit{u0}_{1} \mathit{u0}_{0} + \left(-\frac{2}{3}\right) \mathit{u0}_{1} \mathit{u1}_{1} + \frac{2}{3} \mathit{u1}_{3} \mathit{u1}_{0} + \frac{4}{3} \mathit{u1}_{2} \mathit{u1}_{1} + \left(-\frac{1}{3}\right) \mathit{u0}_{4} + \frac{2}{9} \mathit{u1}_{5}\\f_{1,1} & = & \left(-\frac{4}{3}\right) \mathit{u0}_{1} \mathit{u1}_{0} + \left(-\frac{4}{3}\right) \mathit{u0}_{0} \mathit{u1}_{1} + \frac{2}{3} \mathit{u1}_{2} \mathit{u1}_{0} + \frac{2}{3} \mathit{u1}_{1}^{2} + \left(-\frac{2}{3}\right) \mathit{u0}_{3} + \frac{1}{3} \mathit{u1}_{4}\end{array}\right.

...

P_9 = 1 \mathit{u0}_{2} \mathit{u1}_{0}^{2} z_{0} + 3 \mathit{u0}_{1} \mathit{u0}_{0} \mathit{u1}_{0} z_{0} + 1 \mathit{u0}_{1} \mathit{u1}_{1} \mathit{u1}_{0} z_{0} + 3 \mathit{u0}_{1} \mathit{u1}_{0}^{2} z_{1} + 1 \mathit{u0}_{0}^{3} z_{0} + 3 \mathit{u0}_{0}^{2} \mathit{u1}_{0} z_{1} + 3 \mathit{u0}_{0} \mathit{u1}_{1} \mathit{u1}_{0} z_{1} + 3 \mathit{u0}_{0} \mathit{u1}_{0}^{2} z_{2} + 1 \mathit{u1}_{2} \mathit{u1}_{0}^{2} z_{1} + 1 \mathit{u1}_{1}^{2} \mathit{u1}_{0} z_{1} + 3 \mathit{u1}_{1} \mathit{u1}_{0}^{2} z_{2} + 1 \mathit{u1}_{0}^{3} z_{3} + 2 \mathit{u0}_{4} \mathit{u1}_{0} z_{0} + 3 \mathit{u0}_{3} \mathit{u0}_{0} z_{0} + 3 \mathit{u0}_{3} \mathit{u1}_{1} z_{0} + 9 \mathit{u0}_{3} \mathit{u1}_{0} z_{1} + 6 \mathit{u0}_{2} \mathit{u0}_{1} z_{0} + 9 \mathit{u0}_{2} \mathit{u0}_{0} z_{1} + 3 \mathit{u0}_{2} \mathit{u1}_{2} z_{0} + 12 \mathit{u0}_{2} \mathit{u1}_{1} z_{1} + 15 \mathit{u0}_{2} \mathit{u1}_{0} z_{2} + 6 \mathit{u0}_{1}^{2} z_{1} + 9 \mathit{u0}_{1} \mathit{u0}_{0} z_{2} + 1 \mathit{u0}_{1} \mathit{u1}_{3} z_{0} + 9 \mathit{u0}_{1} \mathit{u1}_{2} z_{1} + 15 \mathit{u0}_{1} \mathit{u1}_{1} z_{2} + 12 \mathit{u0}_{1} \mathit{u1}_{0} z_{3} + 3 \mathit{u0}_{0}^{2} z_{3} + 3 \mathit{u0}_{0} \mathit{u1}_{3} z_{1} + 9 \mathit{u0}_{0} \mathit{u1}_{2} z_{2} + 9 \mathit{u0}_{0} \mathit{u1}_{1} z_{3} + 6 \mathit{u0}_{0} \mathit{u1}_{0} z_{4} + 2 \mathit{u1}_{4} \mathit{u1}_{0} z_{1} + 4 \mathit{u1}_{3} \mathit{u1}_{1} z_{1} + 9 \mathit{u1}_{3} \mathit{u1}_{0} z_{2} + 3 \mathit{u1}_{2}^{2} z_{1} + 15 \mathit{u1}_{2} \mathit{u1}_{1} z_{2} + 15 \mathit{u1}_{2} \mathit{u1}_{0} z_{3} + 9 \mathit{u1}_{1}^{2} z_{3} + 12 \mathit{u1}_{1} \mathit{u1}_{0} z_{4} + 3 \mathit{u1}_{0}^{2} z_{5} + 1 \mathit{u0}_{6} z_{0} + 6 \mathit{u0}_{5} z_{1} + 15 \mathit{u0}_{4} z_{2} + 21 \mathit{u0}_{3} z_{3} + 18 \mathit{u0}_{2} z_{4} + 9 \mathit{u0}_{1} z_{5} + 3 \mathit{u0}_{0} z_{6} + 1 \mathit{u1}_{6} z_{1} + 6 \mathit{u1}_{5} z_{2} + 15 \mathit{u1}_{4} z_{3} + 21 \mathit{u1}_{3} z_{4} + 18 \mathit{u1}_{2} z_{5} + 9 \mathit{u1}_{1} z_{6} + 3 \mathit{u1}_{0} z_{7} + 1 z_{9}

... This commutes with L
Number of ansatz constants: 105
[[322/81, 0, -14/81, 0, 140/27, -140/729, -14/81, 238/27, 70/81, 280/81, 0, 28/27, -16/243, -14/729, 70/81, -14/81, 196/81, -154/243, 280/81, -98/243, 518/81, 280/81, -14/27, 82/81, 392/81, -2/243, 0, 140/81, -70/243, -70/243, 518/81, 280/81, 70/81, 70/243, 140/81, 70/27], [280/27, 140/81, 280/81, 70/243, 140/243, 490/81, 1022/81, 140/27, 1022/81, 82/81, 476/81, 700/27, 70/81, 2023/81, 140/81, 280/27, 280/27, 70/27, 196/81, 566/81, 70/27, 203/9, 350/27, 140/27], [140/27, 1505/81, 1022/81, 21, 350/27, 560/27, 875/27, 280/27, 35/243, 560/81, 245/9, 385/27, 70/9, 2135/81, 566/81, 1085/27, 490/81], [21, 140/3, 973/27, 245/9, 560/27, 140/27, 70/9, 910/27, 280/9, 140/9], [973/27, 175/9, 35/9, 140/9, 350/9, 140/81, 280/9, 175/9], [350/9, 70/9, 175/9, 245/9], [35/9, 35/3, 245/9], [10/3, 35/3], [10/3]]
Generator for the z's in Ring of operator polynomials in (u0, u1, z) over Differential Ring [[Rational Field], (0,)]
---------------------

P_10 = \left(-\frac{14}{729}\right) \mathit{u1}_{0}^{5} z_{0} + \frac{70}{243} \mathit{u0}_{1} \mathit{u1}_{0}^{3} z_{0} + \frac{70}{81} \mathit{u0}_{0}^{2} \mathit{u1}_{0}^{2} z_{0} + \frac{140}{243} \mathit{u0}_{0} \mathit{u1}_{0}^{3} z_{1} + \left(-\frac{140}{729}\right) \mathit{u1}_{2} \mathit{u1}_{0}^{3} z_{0} + \left(-\frac{70}{243}\right) \mathit{u1}_{1}^{2} \mathit{u1}_{0}^{2} z_{0} + \frac{70}{243} \mathit{u1}_{1} \mathit{u1}_{0}^{3} z_{1} + \frac{35}{243} \mathit{u1}_{0}^{4} z_{2} + \frac{140}{81} \mathit{u0}_{3} \mathit{u1}_{0}^{2} z_{0} + \frac{140}{27} \mathit{u0}_{2} \mathit{u0}_{0} \mathit{u1}_{0} z_{0} + \frac{280}{81} \mathit{u0}_{2} \mathit{u1}_{1} \mathit{u1}_{0} z_{0} + \frac{490}{81} \mathit{u0}_{2} \mathit{u1}_{0}^{2} z_{1} + \frac{280}{81} \mathit{u0}_{1}^{2} \mathit{u1}_{0} z_{0} + \frac{70}{27} \mathit{u0}_{1} \mathit{u0}_{0}^{2} z_{0} + \frac{280}{81} \mathit{u0}_{1} \mathit{u0}_{0} \mathit{u1}_{1} z_{0} + \frac{280}{27} \mathit{u0}_{1} \mathit{u0}_{0} \mathit{u1}_{0} z_{1} + \frac{140}{81} \mathit{u0}_{1} \mathit{u1}_{2} \mathit{u1}_{0} z_{0} + \frac{70}{81} \mathit{u0}_{1} \mathit{u1}_{1}^{2} z_{0} + \frac{280}{27} \mathit{u0}_{1} \mathit{u1}_{1} \mathit{u1}_{0} z_{1} + \frac{70}{9} \mathit{u0}_{1} \mathit{u1}_{0}^{2} z_{2} + \frac{140}{81} \mathit{u0}_{0}^{3} z_{1} + \frac{70}{81} \mathit{u0}_{0}^{2} \mathit{u1}_{2} z_{0} + \frac{70}{27} \mathit{u0}_{0}^{2} \mathit{u1}_{1} z_{1} + \frac{140}{27} \mathit{u0}_{0}^{2} \mathit{u1}_{0} z_{2} + \frac{140}{27} \mathit{u0}_{0} \mathit{u1}_{2} \mathit{u1}_{0} z_{1} + \frac{70}{27} \mathit{u0}_{0} \mathit{u1}_{1}^{2} z_{1} + \frac{280}{27} \mathit{u0}_{0} \mathit{u1}_{1} \mathit{u1}_{0} z_{2} + \frac{140}{27} \mathit{u0}_{0} \mathit{u1}_{0}^{2} z_{3} + \left(-\frac{14}{81}\right) \mathit{u1}_{4} \mathit{u1}_{0}^{2} z_{0} + \left(-\frac{154}{243}\right) \mathit{u1}_{3} \mathit{u1}_{1} \mathit{u1}_{0} z_{0} + \frac{140}{81} \mathit{u1}_{3} \mathit{u1}_{0}^{2} z_{1} + \left(-\frac{98}{243}\right) \mathit{u1}_{2}^{2} \mathit{u1}_{0} z_{0} + \left(-\frac{14}{27}\right) \mathit{u1}_{2} \mathit{u1}_{1}^{2} z_{0} + \frac{140}{27} \mathit{u1}_{2} \mathit{u1}_{1} \mathit{u1}_{0} z_{1} + \frac{490}{81} \mathit{u1}_{2} \mathit{u1}_{0}^{2} z_{2} + \frac{70}{81} \mathit{u1}_{1}^{3} z_{1} + \frac{560}{81} \mathit{u1}_{1}^{2} \mathit{u1}_{0} z_{2} + \frac{70}{9} \mathit{u1}_{1} \mathit{u1}_{0}^{2} z_{3} + \frac{140}{81} \mathit{u1}_{0}^{3} z_{4} + \frac{196}{81} \mathit{u0}_{5} \mathit{u1}_{0} z_{0} + \frac{280}{81} \mathit{u0}_{4} \mathit{u0}_{0} z_{0} + \frac{392}{81} \mathit{u0}_{4} \mathit{u1}_{1} z_{0} + \frac{1022}{81} \mathit{u0}_{4} \mathit{u1}_{0} z_{1} + \frac{238}{27} \mathit{u0}_{3} \mathit{u0}_{1} z_{0} + \frac{350}{27} \mathit{u0}_{3} \mathit{u0}_{0} z_{1} + \frac{518}{81} \mathit{u0}_{3} \mathit{u1}_{2} z_{0} + \frac{203}{9} \mathit{u0}_{3} \mathit{u1}_{1} z_{1} + \frac{245}{9} \mathit{u0}_{3} \mathit{u1}_{0} z_{2} + \frac{518}{81} \mathit{u0}_{2}^{2} z_{0} + \frac{700}{27} \mathit{u0}_{2} \mathit{u0}_{1} z_{1} + \frac{560}{27} \mathit{u0}_{2} \mathit{u0}_{0} z_{2} + \frac{322}{81} \mathit{u0}_{2} \mathit{u1}_{3} z_{0} + \frac{2023}{81} \mathit{u0}_{2} \mathit{u1}_{2} z_{1} + \frac{1085}{27} \mathit{u0}_{2} \mathit{u1}_{1} z_{2} + \frac{280}{9} \mathit{u0}_{2} \mathit{u1}_{0} z_{3} + \frac{385}{27} \mathit{u0}_{1}^{2} z_{2} + \frac{140}{9} \mathit{u0}_{1} \mathit{u0}_{0} z_{3} + \frac{28}{27} \mathit{u0}_{1} \mathit{u1}_{4} z_{0} + \frac{1022}{81} \mathit{u0}_{1} \mathit{u1}_{3} z_{1} + \frac{875}{27} \mathit{u0}_{1} \mathit{u1}_{2} z_{2} + \frac{910}{27} \mathit{u0}_{1} \mathit{u1}_{1} z_{3} + \frac{175}{9} \mathit{u0}_{1} \mathit{u1}_{0} z_{4} + \frac{35}{9} \mathit{u0}_{0}^{2} z_{4} + \frac{280}{81} \mathit{u0}_{0} \mathit{u1}_{4} z_{1} + \frac{350}{27} \mathit{u0}_{0} \mathit{u1}_{3} z_{2} + \frac{560}{27} \mathit{u0}_{0} \mathit{u1}_{2} z_{3} + \frac{140}{9} \mathit{u0}_{0} \mathit{u1}_{1} z_{4} + \frac{70}{9} \mathit{u0}_{0} \mathit{u1}_{0} z_{5} + \lef

\left\{\begin{array}{rcl}f_{3,0} & = & \left(-\frac{70}{729}\right) \mathit{u1}_{1} \mathit{u1}_{0}^{5} + \frac{35}{243} \mathit{u0}_{2} \mathit{u1}_{0}^{4} + \frac{280}{243} \mathit{u0}_{1} \mathit{u0}_{0} \mathit{u1}_{0}^{3} + \frac{140}{243} \mathit{u0}_{1} \mathit{u1}_{1} \mathit{u1}_{0}^{3} + \frac{140}{81} \mathit{u0}_{0}^{2} \mathit{u1}_{1} \mathit{u1}_{0}^{2} + \left(-\frac{70}{243}\right) \mathit{u1}_{3} \mathit{u1}_{0}^{4} + \left(-\frac{560}{243}\right) \mathit{u1}_{2} \mathit{u1}_{1} \mathit{u1}_{0}^{3} + \left(-\frac{140}{81}\right) \mathit{u1}_{1}^{3} \mathit{u1}_{0}^{2} + \frac{70}{243} \mathit{u0}_{4} \mathit{u1}_{0}^{3} + \frac{140}{81} \mathit{u0}_{3} \mathit{u0}_{0} \mathit{u1}_{0}^{2} + \frac{140}{81} \mathit{u0}_{3} \mathit{u1}_{1} \mathit{u1}_{0}^{2} + \frac{280}{81} \mathit{u0}_{2} \mathit{u0}_{1} \mathit{u1}_{0}^{2} + \left(-\frac{70}{27}\right) \mathit{u0}_{2} \mathit{u0}_{0}^{2} \mathit{u1}_{0} + \frac{700}{81} \mathit{u0}_{2} \mathit{u0}_{0} \mathit{u1}_{1} \mathit{u1}_{0} + \frac{140}{81} \mathit{u0}_{2} \mathit{u1}_{2} \mathit{u1}_{0}^{2} + \frac{70}{27} \mathit{u0}_{2} \mathit{u1}_{1}^{2} \mathit{u1}_{0} + \left(-\frac{140}{27}\right) \mathit{u0}_{1}^{2} \mathit{u0}_{0} \mathit{u1}_{0} + \frac{560}{81} \mathit{u0}_{1}^{2} \mathit{u1}_{1} \mathit{u1}_{0} + \left(-\frac{140}{81}\right) \mathit{u0}_{1} \mathit{u0}_{0}^{3} + \left(-\frac{70}{27}\right) \mathit{u0}_{1} \mathit{u0}_{0}^{2} \mathit{u1}_{1} + \frac{280}{27} \mathit{u0}_{1} \mathit{u0}_{0} \mathit{u1}_{2} \mathit{u1}_{0} + \frac{70}{9} \mathit{u0}_{1} \mathit{u0}_{0} \mathit{u1}_{1}^{2} + \frac{70}{81} \mathit{u0}_{1} \mathit{u1}_{3} \mathit{u1}_{0}^{2} + \frac{280}{81} \mathit{u0}_{1} \mathit{u1}_{2} \mathit{u1}_{1} \mathit{u1}_{0} + \frac{70}{81} \mathit{u0}_{1} \mathit{u1}_{1}^{3} + \frac{70}{27} \mathit{u0}_{0}^{2} \mathit{u1}_{3} \mathit{u1}_{0} + \frac{140}{27} \mathit{u0}_{0}^{2} \mathit{u1}_{2} \mathit{u1}_{1} + \left(-\frac{266}{729}\right) \mathit{u1}_{5} \mathit{u1}_{0}^{3} + \left(-\frac{266}{81}\right) \mathit{u1}_{4} \mathit{u1}_{1} \mathit{u1}_{0}^{2} + \left(-\frac{1330}{243}\right) \mathit{u1}_{3} \mathit{u1}_{2} \mathit{u1}_{0}^{2} + \left(-\frac{700}{81}\right) \mathit{u1}_{3} \mathit{u1}_{1}^{2} \mathit{u1}_{0} + \left(-\frac{2870}{243}\right) \mathit{u1}_{2}^{2} \mathit{u1}_{1} \mathit{u1}_{0} + \left(-\frac{1540}{243}\right) \mathit{u1}_{2} \mathit{u1}_{1}^{3} + \frac{7}{27} \mathit{u0}_{6} \mathit{u1}_{0}^{2} + \frac{70}{81} \mathit{u0}_{5} \mathit{u0}_{0} \mathit{u1}_{0} + \frac{133}{81} \mathit{u0}_{5} \mathit{u1}_{1} \mathit{u1}_{0} + \frac{217}{81} \mathit{u0}_{4} \mathit{u0}_{1} \mathit{u1}_{0} + \left(-\frac{35}{27}\right) \mathit{u0}_{4} \mathit{u0}_{0}^{2} + \frac{280}{81} \mathit{u0}_{4} \mathit{u0}_{0} \mathit{u1}_{1} + \frac{70}{27} \mathit{u0}_{4} \mathit{u1}_{2} \mathit{u1}_{0} + \frac{175}{81} \mathit{u0}_{4} \mathit{u1}_{1}^{2} + \frac{385}{81} \mathit{u0}_{3} \mathit{u0}_{2} \mathit{u1}_{0} + \left(-\frac{70}{9}\right) \mathit{u0}_{3} \mathit{u0}_{1} \mathit{u0}_{0} + \frac{763}{81} \mathit{u0}_{3} \mathit{u0}_{1} \mathit{u1}_{1} + \frac{560}{81} \mathit{u0}_{3} \mathit{u0}_{0} \mathit{u1}_{2} + \frac{175}{81} \mathit{u0}_{3} \mathit{u1}_{3} \mathit{u1}_{0} + \frac{140}{27} \mathit{u0}_{3} \mathit{u1}_{2} \mathit{u1}_{1} + \left(-\frac{140}{27}\right) \mathit{u0}_{2}^{2} \mathit{u0}_{0} + \frac{175}{27} \mathit{u0}_{2}^{2} \mathit{u1}_{1} + \left(-\frac{245}{27}\right) \mathit{u0}_{2} \mathit{u0}_{1}^{2} + \frac{1232}{81} \mathit{u0}_{2} \mathit{u0}_{1} \mathit{u1}_{2} + \frac{70}{9} \mathit{u0}_{2} \mathit{u0}_{0} \mathit{u1}_{3} + \frac{28}{27} \mathit{u0}_{2} \mathit{u1}_{4} \mathit{u1}_{0} + \frac{245}{81} \mathit{u0}_{2} \mathit{u1}_{3} \mathit{u1}_{1} + \frac{175}{81} \mathit{u0}_{2} \mathit{u1}_{2}^{2} + \frac{518}{81} \mathit{u0}_{1}^{2} \mathit{u1}_{3} + \frac{140}{27} \mathit{u0}_{1} \mathit{u0}_{0} \mathit{u1}_{4} + \frac{28}{81} \mathit{u0}_{1} \mathit{u1}_{5} \mathit{u1}_{0} + \frac{28}{2

CPU times: user 22.3 s, sys: 58.4 ms, total: 22.4 s
Wall time: 22.4 s


In [113]:
%time seq=get_sequence(3, 10, quasi_commutting_generic_diff)

-------------------------------------------------------------------------------------------------------------------
Computed the almost commutator of order 3


P_3 = 1 \mathit{u0}_{0} z_{0} + 1 \mathit{u1}_{0} z_{1} + 1 z_{3}

... This commutes with L
-------------------------------------------------------------------------------------------------------------------
Computed the almost commutator of order 4


P_4 = \frac{2}{9} \mathit{u1}_{0}^{2} z_{0} + \frac{2}{3} \mathit{u0}_{1} z_{0} + \frac{4}{3} \mathit{u0}_{0} z_{1} + \frac{2}{9} \mathit{u1}_{2} z_{0} + \frac{2}{3} \mathit{u1}_{1} z_{1} + \frac{4}{3} \mathit{u1}_{0} z_{2} + 1 z_{4}

\left\{\begin{array}{rcl}f_{1,0} & = & \frac{4}{9} \mathit{u1}_{1} \mathit{u1}_{0}^{2} + \left(-\frac{2}{3}\right) \mathit{u0}_{2} \mathit{u1}_{0} + \left(-\frac{4}{3}\right) \mathit{u0}_{1} \mathit{u0}_{0} + \left(-\frac{2}{3}\right) \mathit{u0}_{1} \mathit{u1}_{1} + \frac{2}{3} \mathit{u1}_{3} \mathit{u1}_{0} + \frac{4}{3} \mathit{u1}_{2} \mathit{u1}_{1} + \left(-\frac{1}{3}\right) \mathit{u0}_{4} + \frac{2}{9} \mathit{u1}_{5}\\f_{1,1} & = & \left(-\frac{4}{3}\right) \mathit{u0}_{1} \mathit{u1}_{0} + \left(-\frac{4}{3}\right) \mathit{u0}_{0} \mathit{u1}_{1} + \frac{2}{3} \mathit{u1}_{2} \mathit{u1}_{0} + \frac{2}{3} \mathit{u1}_{1}^{2} + \left(-\frac{2}{3}\right) \mathit{u0}_{3} + \frac{1}{3} \mathit{u1}_{4}\end{array}\right.

...

P_9 = 1 \mathit{u0}_{2} \mathit{u1}_{0}^{2} z_{0} + 3 \mathit{u0}_{1} \mathit{u0}_{0} \mathit{u1}_{0} z_{0} + 1 \mathit{u0}_{1} \mathit{u1}_{1} \mathit{u1}_{0} z_{0} + 3 \mathit{u0}_{1} \mathit{u1}_{0}^{2} z_{1} + 1 \mathit{u0}_{0}^{3} z_{0} + 3 \mathit{u0}_{0}^{2} \mathit{u1}_{0} z_{1} + 3 \mathit{u0}_{0} \mathit{u1}_{1} \mathit{u1}_{0} z_{1} + 3 \mathit{u0}_{0} \mathit{u1}_{0}^{2} z_{2} + 1 \mathit{u1}_{2} \mathit{u1}_{0}^{2} z_{1} + 1 \mathit{u1}_{1}^{2} \mathit{u1}_{0} z_{1} + 3 \mathit{u1}_{1} \mathit{u1}_{0}^{2} z_{2} + 1 \mathit{u1}_{0}^{3} z_{3} + 2 \mathit{u0}_{4} \mathit{u1}_{0} z_{0} + 3 \mathit{u0}_{3} \mathit{u0}_{0} z_{0} + 3 \mathit{u0}_{3} \mathit{u1}_{1} z_{0} + 9 \mathit{u0}_{3} \mathit{u1}_{0} z_{1} + 6 \mathit{u0}_{2} \mathit{u0}_{1} z_{0} + 9 \mathit{u0}_{2} \mathit{u0}_{0} z_{1} + 3 \mathit{u0}_{2} \mathit{u1}_{2} z_{0} + 12 \mathit{u0}_{2} \mathit{u1}_{1} z_{1} + 15 \mathit{u0}_{2} \mathit{u1}_{0} z_{2} + 6 \mathit{u0}_{1}^{2} z_{1} + 9 \mathit{u0}_{1} \mathit{u0}_{0} z_{2} + 1 \mathit{u0}_{1} \mathit{u1}_{3} z_{0} + 9 \mathit{u0}_{1} \mathit{u1}_{2} z_{1} + 15 \mathit{u0}_{1} \mathit{u1}_{1} z_{2} + 12 \mathit{u0}_{1} \mathit{u1}_{0} z_{3} + 3 \mathit{u0}_{0}^{2} z_{3} + 3 \mathit{u0}_{0} \mathit{u1}_{3} z_{1} + 9 \mathit{u0}_{0} \mathit{u1}_{2} z_{2} + 9 \mathit{u0}_{0} \mathit{u1}_{1} z_{3} + 6 \mathit{u0}_{0} \mathit{u1}_{0} z_{4} + 2 \mathit{u1}_{4} \mathit{u1}_{0} z_{1} + 4 \mathit{u1}_{3} \mathit{u1}_{1} z_{1} + 9 \mathit{u1}_{3} \mathit{u1}_{0} z_{2} + 3 \mathit{u1}_{2}^{2} z_{1} + 15 \mathit{u1}_{2} \mathit{u1}_{1} z_{2} + 15 \mathit{u1}_{2} \mathit{u1}_{0} z_{3} + 9 \mathit{u1}_{1}^{2} z_{3} + 12 \mathit{u1}_{1} \mathit{u1}_{0} z_{4} + 3 \mathit{u1}_{0}^{2} z_{5} + 1 \mathit{u0}_{6} z_{0} + 6 \mathit{u0}_{5} z_{1} + 15 \mathit{u0}_{4} z_{2} + 21 \mathit{u0}_{3} z_{3} + 18 \mathit{u0}_{2} z_{4} + 9 \mathit{u0}_{1} z_{5} + 3 \mathit{u0}_{0} z_{6} + 1 \mathit{u1}_{6} z_{1} + 6 \mathit{u1}_{5} z_{2} + 15 \mathit{u1}_{4} z_{3} + 21 \mathit{u1}_{3} z_{4} + 18 \mathit{u1}_{2} z_{5} + 9 \mathit{u1}_{1} z_{6} + 3 \mathit{u1}_{0} z_{7} + 1 z_{9}

... This commutes with L
-------------------------------------------------------------------------------------------------------------------
Computed the almost commutator of order 10


P_10 = \left(-\frac{14}{729}\right) \mathit{u1}_{0}^{5} z_{0} + \frac{70}{243} \mathit{u0}_{1} \mathit{u1}_{0}^{3} z_{0} + \frac{70}{81} \mathit{u0}_{0}^{2} \mathit{u1}_{0}^{2} z_{0} + \frac{140}{243} \mathit{u0}_{0} \mathit{u1}_{0}^{3} z_{1} + \left(-\frac{140}{729}\right) \mathit{u1}_{2} \mathit{u1}_{0}^{3} z_{0} + \left(-\frac{70}{243}\right) \mathit{u1}_{1}^{2} \mathit{u1}_{0}^{2} z_{0} + \frac{70}{243} \mathit{u1}_{1} \mathit{u1}_{0}^{3} z_{1} + \frac{35}{243} \mathit{u1}_{0}^{4} z_{2} + \frac{140}{81} \mathit{u0}_{3} \mathit{u1}_{0}^{2} z_{0} + \frac{140}{27} \mathit{u0}_{2} \mathit{u0}_{0} \mathit{u1}_{0} z_{0} + \frac{280}{81} \mathit{u0}_{2} \mathit{u1}_{1} \mathit{u1}_{0} z_{0} + \frac{490}{81} \mathit{u0}_{2} \mathit{u1}_{0}^{2} z_{1} + \frac{280}{81} \mathit{u0}_{1}^{2} \mathit{u1}_{0} z_{0} + \frac{70}{27} \mathit{u0}_{1} \mathit{u0}_{0}^{2} z_{0} + \frac{280}{81} \mathit{u0}_{1} \mathit{u0}_{0} \mathit{u1}_{1} z_{0} + \frac{280}{27} \mathit{u0}_{1} \mathit{u0}_{0} \mathit{u1}_{0} z_{1} + \frac{140}{81} \mathit{u0}_{1} \mathit{u1}_{2} \mathit{u1}_{0} z_{0} + \frac{70}{81} \mathit{u0}_{1} \mathit{u1}_{1}^{2} z_{0} + \frac{280}{27} \mathit{u0}_{1} \mathit{u1}_{1} \mathit{u1}_{0} z_{1} + \frac{70}{9} \mathit{u0}_{1} \mathit{u1}_{0}^{2} z_{2} + \frac{140}{81} \mathit{u0}_{0}^{3} z_{1} + \frac{70}{81} \mathit{u0}_{0}^{2} \mathit{u1}_{2} z_{0} + \frac{70}{27} \mathit{u0}_{0}^{2} \mathit{u1}_{1} z_{1} + \frac{140}{27} \mathit{u0}_{0}^{2} \mathit{u1}_{0} z_{2} + \frac{140}{27} \mathit{u0}_{0} \mathit{u1}_{2} \mathit{u1}_{0} z_{1} + \frac{70}{27} \mathit{u0}_{0} \mathit{u1}_{1}^{2} z_{1} + \frac{280}{27} \mathit{u0}_{0} \mathit{u1}_{1} \mathit{u1}_{0} z_{2} + \frac{140}{27} \mathit{u0}_{0} \mathit{u1}_{0}^{2} z_{3} + \left(-\frac{14}{81}\right) \mathit{u1}_{4} \mathit{u1}_{0}^{2} z_{0} + \left(-\frac{154}{243}\right) \mathit{u1}_{3} \mathit{u1}_{1} \mathit{u1}_{0} z_{0} + \frac{140}{81} \mathit{u1}_{3} \mathit{u1}_{0}^{2} z_{1} + \left(-\frac{98}{243}\right) \mathit{u1}_{2}^{2} \mathit{u1}_{0} z_{0} + \left(-\frac{14}{27}\right) \mathit{u1}_{2} \mathit{u1}_{1}^{2} z_{0} + \frac{140}{27} \mathit{u1}_{2} \mathit{u1}_{1} \mathit{u1}_{0} z_{1} + \frac{490}{81} \mathit{u1}_{2} \mathit{u1}_{0}^{2} z_{2} + \frac{70}{81} \mathit{u1}_{1}^{3} z_{1} + \frac{560}{81} \mathit{u1}_{1}^{2} \mathit{u1}_{0} z_{2} + \frac{70}{9} \mathit{u1}_{1} \mathit{u1}_{0}^{2} z_{3} + \frac{140}{81} \mathit{u1}_{0}^{3} z_{4} + \frac{196}{81} \mathit{u0}_{5} \mathit{u1}_{0} z_{0} + \frac{280}{81} \mathit{u0}_{4} \mathit{u0}_{0} z_{0} + \frac{392}{81} \mathit{u0}_{4} \mathit{u1}_{1} z_{0} + \frac{1022}{81} \mathit{u0}_{4} \mathit{u1}_{0} z_{1} + \frac{238}{27} \mathit{u0}_{3} \mathit{u0}_{1} z_{0} + \frac{350}{27} \mathit{u0}_{3} \mathit{u0}_{0} z_{1} + \frac{518}{81} \mathit{u0}_{3} \mathit{u1}_{2} z_{0} + \frac{203}{9} \mathit{u0}_{3} \mathit{u1}_{1} z_{1} + \frac{245}{9} \mathit{u0}_{3} \mathit{u1}_{0} z_{2} + \frac{518}{81} \mathit{u0}_{2}^{2} z_{0} + \frac{700}{27} \mathit{u0}_{2} \mathit{u0}_{1} z_{1} + \frac{560}{27} \mathit{u0}_{2} \mathit{u0}_{0} z_{2} + \frac{322}{81} \mathit{u0}_{2} \mathit{u1}_{3} z_{0} + \frac{2023}{81} \mathit{u0}_{2} \mathit{u1}_{2} z_{1} + \frac{1085}{27} \mathit{u0}_{2} \mathit{u1}_{1} z_{2} + \frac{280}{9} \mathit{u0}_{2} \mathit{u1}_{0} z_{3} + \frac{385}{27} \mathit{u0}_{1}^{2} z_{2} + \frac{140}{9} \mathit{u0}_{1} \mathit{u0}_{0} z_{3} + \frac{28}{27} \mathit{u0}_{1} \mathit{u1}_{4} z_{0} + \frac{1022}{81} \mathit{u0}_{1} \mathit{u1}_{3} z_{1} + \frac{875}{27} \mathit{u0}_{1} \mathit{u1}_{2} z_{2} + \frac{910}{27} \mathit{u0}_{1} \mathit{u1}_{1} z_{3} + \frac{175}{9} \mathit{u0}_{1} \mathit{u1}_{0} z_{4} + \frac{35}{9} \mathit{u0}_{0}^{2} z_{4} + \frac{280}{81} \mathit{u0}_{0} \mathit{u1}_{4} z_{1} + \frac{350}{27} \mathit{u0}_{0} \mathit{u1}_{3} z_{2} + \frac{560}{27} \mathit{u0}_{0} \mathit{u1}_{2} z_{3} + \frac{140}{9} \mathit{u0}_{0} \mathit{u1}_{1} z_{4} + \frac{70}{9} \mathit{u0}_{0} \mathit{u1}_{0} z_{5} + \lef

\left\{\begin{array}{rcl}f_{3,0} & = & \left(-\frac{70}{729}\right) \mathit{u1}_{1} \mathit{u1}_{0}^{5} + \frac{35}{243} \mathit{u0}_{2} \mathit{u1}_{0}^{4} + \frac{280}{243} \mathit{u0}_{1} \mathit{u0}_{0} \mathit{u1}_{0}^{3} + \frac{140}{243} \mathit{u0}_{1} \mathit{u1}_{1} \mathit{u1}_{0}^{3} + \frac{140}{81} \mathit{u0}_{0}^{2} \mathit{u1}_{1} \mathit{u1}_{0}^{2} + \left(-\frac{70}{243}\right) \mathit{u1}_{3} \mathit{u1}_{0}^{4} + \left(-\frac{560}{243}\right) \mathit{u1}_{2} \mathit{u1}_{1} \mathit{u1}_{0}^{3} + \left(-\frac{140}{81}\right) \mathit{u1}_{1}^{3} \mathit{u1}_{0}^{2} + \frac{70}{243} \mathit{u0}_{4} \mathit{u1}_{0}^{3} + \frac{140}{81} \mathit{u0}_{3} \mathit{u0}_{0} \mathit{u1}_{0}^{2} + \frac{140}{81} \mathit{u0}_{3} \mathit{u1}_{1} \mathit{u1}_{0}^{2} + \frac{280}{81} \mathit{u0}_{2} \mathit{u0}_{1} \mathit{u1}_{0}^{2} + \left(-\frac{70}{27}\right) \mathit{u0}_{2} \mathit{u0}_{0}^{2} \mathit{u1}_{0} + \frac{700}{81} \mathit{u0}_{2} \mathit{u0}_{0} \mathit{u1}_{1} \mathit{u1}_{0} + \frac{140}{81} \mathit{u0}_{2} \mathit{u1}_{2} \mathit{u1}_{0}^{2} + \frac{70}{27} \mathit{u0}_{2} \mathit{u1}_{1}^{2} \mathit{u1}_{0} + \left(-\frac{140}{27}\right) \mathit{u0}_{1}^{2} \mathit{u0}_{0} \mathit{u1}_{0} + \frac{560}{81} \mathit{u0}_{1}^{2} \mathit{u1}_{1} \mathit{u1}_{0} + \left(-\frac{140}{81}\right) \mathit{u0}_{1} \mathit{u0}_{0}^{3} + \left(-\frac{70}{27}\right) \mathit{u0}_{1} \mathit{u0}_{0}^{2} \mathit{u1}_{1} + \frac{280}{27} \mathit{u0}_{1} \mathit{u0}_{0} \mathit{u1}_{2} \mathit{u1}_{0} + \frac{70}{9} \mathit{u0}_{1} \mathit{u0}_{0} \mathit{u1}_{1}^{2} + \frac{70}{81} \mathit{u0}_{1} \mathit{u1}_{3} \mathit{u1}_{0}^{2} + \frac{280}{81} \mathit{u0}_{1} \mathit{u1}_{2} \mathit{u1}_{1} \mathit{u1}_{0} + \frac{70}{81} \mathit{u0}_{1} \mathit{u1}_{1}^{3} + \frac{70}{27} \mathit{u0}_{0}^{2} \mathit{u1}_{3} \mathit{u1}_{0} + \frac{140}{27} \mathit{u0}_{0}^{2} \mathit{u1}_{2} \mathit{u1}_{1} + \left(-\frac{266}{729}\right) \mathit{u1}_{5} \mathit{u1}_{0}^{3} + \left(-\frac{266}{81}\right) \mathit{u1}_{4} \mathit{u1}_{1} \mathit{u1}_{0}^{2} + \left(-\frac{1330}{243}\right) \mathit{u1}_{3} \mathit{u1}_{2} \mathit{u1}_{0}^{2} + \left(-\frac{700}{81}\right) \mathit{u1}_{3} \mathit{u1}_{1}^{2} \mathit{u1}_{0} + \left(-\frac{2870}{243}\right) \mathit{u1}_{2}^{2} \mathit{u1}_{1} \mathit{u1}_{0} + \left(-\frac{1540}{243}\right) \mathit{u1}_{2} \mathit{u1}_{1}^{3} + \frac{7}{27} \mathit{u0}_{6} \mathit{u1}_{0}^{2} + \frac{70}{81} \mathit{u0}_{5} \mathit{u0}_{0} \mathit{u1}_{0} + \frac{133}{81} \mathit{u0}_{5} \mathit{u1}_{1} \mathit{u1}_{0} + \frac{217}{81} \mathit{u0}_{4} \mathit{u0}_{1} \mathit{u1}_{0} + \left(-\frac{35}{27}\right) \mathit{u0}_{4} \mathit{u0}_{0}^{2} + \frac{280}{81} \mathit{u0}_{4} \mathit{u0}_{0} \mathit{u1}_{1} + \frac{70}{27} \mathit{u0}_{4} \mathit{u1}_{2} \mathit{u1}_{0} + \frac{175}{81} \mathit{u0}_{4} \mathit{u1}_{1}^{2} + \frac{385}{81} \mathit{u0}_{3} \mathit{u0}_{2} \mathit{u1}_{0} + \left(-\frac{70}{9}\right) \mathit{u0}_{3} \mathit{u0}_{1} \mathit{u0}_{0} + \frac{763}{81} \mathit{u0}_{3} \mathit{u0}_{1} \mathit{u1}_{1} + \frac{560}{81} \mathit{u0}_{3} \mathit{u0}_{0} \mathit{u1}_{2} + \frac{175}{81} \mathit{u0}_{3} \mathit{u1}_{3} \mathit{u1}_{0} + \frac{140}{27} \mathit{u0}_{3} \mathit{u1}_{2} \mathit{u1}_{1} + \left(-\frac{140}{27}\right) \mathit{u0}_{2}^{2} \mathit{u0}_{0} + \frac{175}{27} \mathit{u0}_{2}^{2} \mathit{u1}_{1} + \left(-\frac{245}{27}\right) \mathit{u0}_{2} \mathit{u0}_{1}^{2} + \frac{1232}{81} \mathit{u0}_{2} \mathit{u0}_{1} \mathit{u1}_{2} + \frac{70}{9} \mathit{u0}_{2} \mathit{u0}_{0} \mathit{u1}_{3} + \frac{28}{27} \mathit{u0}_{2} \mathit{u1}_{4} \mathit{u1}_{0} + \frac{245}{81} \mathit{u0}_{2} \mathit{u1}_{3} \mathit{u1}_{1} + \frac{175}{81} \mathit{u0}_{2} \mathit{u1}_{2}^{2} + \frac{518}{81} \mathit{u0}_{1}^{2} \mathit{u1}_{3} + \frac{140}{27} \mathit{u0}_{1} \mathit{u0}_{0} \mathit{u1}_{4} + \frac{28}{81} \mathit{u0}_{1} \mathit{u1}_{5} \mathit{u1}_{0} + \frac{28}{2

CPU times: user 10.1 s, sys: 11.9 ms, total: 10.1 s
Wall time: 10.1 s


## Getting the sequence for operator of order 5

In [115]:
%time seq=get_sequence(5, 10, quasi_commutting_generic_linear)

Number of ansatz constants: 13
[[1, 0, 0, 0, 0, 0], [0, 0, 0, 1], [0, 1], [1]]
Generator for the z's in Ring of operator polynomials in (u0, u1, u2, u3, z) over Differential Ring [[Rational Field], (0,)]
-------------------------------------------------------------------------------------------------------------------
Computed the almost commutator of order 5


P_5 = 1 \mathit{u0}_{0} z_{0} + 1 \mathit{u1}_{0} z_{1} + 1 \mathit{u2}_{0} z_{2} + 1 \mathit{u3}_{0} z_{3} + 1 z_{5}

... This commutes with L
Number of ansatz constants: 24
[[-4/125, -1/5, -6/25, -3/25, 6/25, 2/5, 3/25, 3/5, -3/25, -1/25, -1/25], [6/5, 2/5, -1/5, -3/25, 6/25, 3/5], [3/5, 3/25, 2/5, 6/5], [3/5, 6/5], [6/5]]
Generator for the z's in Ring of operator polynomials in (u0, u1, u2, u3, z) over Differential Ring [[Rational Field], (0,)]
-------------------------------------------------------------------------------------------------------------------
Computed the almost commutator of order 6


P_6 = \left(-\frac{4}{125}\right) \mathit{u3}_{0}^{3} z_{0} + \frac{6}{25} \mathit{u1}_{0} \mathit{u3}_{0} z_{0} + \left(-\frac{3}{25}\right) \mathit{u2}_{1} \mathit{u3}_{0} z_{0} + \frac{3}{25} \mathit{u2}_{0}^{2} z_{0} + \left(-\frac{6}{25}\right) \mathit{u2}_{0} \mathit{u3}_{1} z_{0} + \frac{6}{25} \mathit{u2}_{0} \mathit{u3}_{0} z_{1} + \left(-\frac{3}{25}\right) \mathit{u3}_{2} \mathit{u3}_{0} z_{0} + \left(-\frac{1}{25}\right) \mathit{u3}_{1}^{2} z_{0} + \left(-\frac{3}{25}\right) \mathit{u3}_{1} \mathit{u3}_{0} z_{1} + \frac{3}{25} \mathit{u3}_{0}^{2} z_{2} + \frac{3}{5} \mathit{u0}_{1} z_{0} + \frac{6}{5} \mathit{u0}_{0} z_{1} + \frac{2}{5} \mathit{u1}_{2} z_{0} + \frac{3}{5} \mathit{u1}_{1} z_{1} + \frac{6}{5} \mathit{u1}_{0} z_{2} + \left(-\frac{1}{5}\right) \mathit{u2}_{3} z_{0} + \frac{2}{5} \mathit{u2}_{2} z_{1} + \frac{3}{5} \mathit{u2}_{1} z_{2} + \frac{6}{5} \mathit{u2}_{0} z_{3} + \left(-\frac{1}{25}\right) \mathit{u3}_{4} z_{0} + \left(-\frac{1}{5}\right) \mathit{u3}_{3} z_{1} + \frac{2}{5} \mathit{u3}_{2} z_{2} + \frac{3}{5} \mathit{u3}_{1} z_{3} + \frac{6}{5} \mathit{u3}_{0} z_{4} + 1 z_{6}

\left\{\begin{array}{rcl}f_{1,0} & = & \left(-\frac{12}{125}\right) \mathit{u1}_{0} \mathit{u3}_{1} \mathit{u3}_{0}^{2} + \left(-\frac{12}{125}\right) \mathit{u2}_{0} \mathit{u3}_{2} \mathit{u3}_{0}^{2} + \left(-\frac{24}{125}\right) \mathit{u2}_{0} \mathit{u3}_{1}^{2} \mathit{u3}_{0} + \left(-\frac{12}{125}\right) \mathit{u3}_{3} \mathit{u3}_{0}^{3} + \left(-\frac{72}{125}\right) \mathit{u3}_{2} \mathit{u3}_{1} \mathit{u3}_{0}^{2} + \left(-\frac{24}{125}\right) \mathit{u3}_{1}^{3} \mathit{u3}_{0} + \left(-\frac{3}{25}\right) \mathit{u0}_{2} \mathit{u3}_{0}^{2} + \left(-\frac{6}{25}\right) \mathit{u0}_{1} \mathit{u2}_{0} \mathit{u3}_{0} + \frac{3}{25} \mathit{u0}_{1} \mathit{u3}_{1} \mathit{u3}_{0} + \frac{6}{25} \mathit{u1}_{3} \mathit{u3}_{0}^{2} + \frac{6}{25} \mathit{u1}_{2} \mathit{u2}_{0} \mathit{u3}_{0} + \frac{18}{25} \mathit{u1}_{2} \mathit{u3}_{1} \mathit{u3}_{0} + \frac{6}{25} \mathit{u1}_{1} \mathit{u1}_{0} \mathit{u3}_{0} + \frac{12}{25} \mathit{u1}_{1} \mathit{u2}_{0} \mathit{u3}_{1} + \frac{18}{25} \mathit{u1}_{1} \mathit{u3}_{2} \mathit{u3}_{0} + \frac{6}{25} \mathit{u1}_{0}^{2} \mathit{u3}_{1} + \left(-\frac{3}{25}\right) \mathit{u1}_{0} \mathit{u2}_{2} \mathit{u3}_{0} + \frac{6}{25} \mathit{u1}_{0} \mathit{u2}_{1} \mathit{u2}_{0} + \left(-\frac{9}{25}\right) \mathit{u1}_{0} \mathit{u2}_{1} \mathit{u3}_{1} + \frac{3}{25} \mathit{u1}_{0} \mathit{u3}_{3} \mathit{u3}_{0} + \left(-\frac{1}{5}\right) \mathit{u1}_{0} \mathit{u3}_{2} \mathit{u3}_{1} + \left(-\frac{3}{25}\right) \mathit{u2}_{4} \mathit{u3}_{0}^{2} + \frac{3}{25} \mathit{u2}_{3} \mathit{u2}_{0} \mathit{u3}_{0} + \left(-\frac{3}{5}\right) \mathit{u2}_{3} \mathit{u3}_{1} \mathit{u3}_{0} + \frac{18}{25} \mathit{u2}_{2} \mathit{u2}_{1} \mathit{u3}_{0} + \frac{6}{25} \mathit{u2}_{2} \mathit{u2}_{0}^{2} + \left(-\frac{12}{25}\right) \mathit{u2}_{2} \mathit{u2}_{0} \mathit{u3}_{1} + \left(-\frac{27}{25}\right) \mathit{u2}_{2} \mathit{u3}_{2} \mathit{u3}_{0} + \frac{6}{25} \mathit{u2}_{1}^{2} \mathit{u2}_{0} + \left(-\frac{3}{5}\right) \mathit{u2}_{1} \mathit{u2}_{0} \mathit{u3}_{2} + \left(-\frac{21}{25}\right) \mathit{u2}_{1} \mathit{u3}_{3} \mathit{u3}_{0} + \left(-\frac{6}{25}\right) \mathit{u2}_{0}^{2} \mathit{u3}_{3} + \left(-\frac{9}{25}\right) \mathit{u2}_{0} \mathit{u3}_{4} \mathit{u3}_{0} + \left(-\frac{8}{25}\right) \mathit{u2}_{0} \mathit{u3}_{3} \mathit{u3}_{1} + \left(-\frac{1}{5}\right) \mathit{u2}_{0} \mathit{u3}_{2}^{2} + \left(-\frac{27}{125}\right) \mathit{u3}_{5} \mathit{u3}_{0}^{2} + \left(-\frac{7}{5}\right) \mathit{u3}_{4} \mathit{u3}_{1} \mathit{u3}_{0} + \left(-\frac{66}{25}\right) \mathit{u3}_{3} \mathit{u3}_{2} \mathit{u3}_{0} + \left(-\frac{48}{25}\right) \mathit{u3}_{3} \mathit{u3}_{1}^{2} + \left(-\frac{72}{25}\right) \mathit{u3}_{2}^{2} \mathit{u3}_{1} + \left(-\frac{3}{5}\right) \mathit{u0}_{4} \mathit{u3}_{0} + \left(-\frac{3}{5}\right) \mathit{u0}_{3} \mathit{u2}_{0} + \left(-\frac{3}{5}\right) \mathit{u0}_{3} \mathit{u3}_{1} + \left(-\frac{3}{5}\right) \mathit{u0}_{2} \mathit{u1}_{0} + \left(-\frac{3}{5}\right) \mathit{u0}_{2} \mathit{u2}_{1} + \left(-\frac{2}{5}\right) \mathit{u0}_{2} \mathit{u3}_{2} + \left(-\frac{6}{5}\right) \mathit{u0}_{1} \mathit{u0}_{0} + \left(-\frac{3}{5}\right) \mathit{u0}_{1} \mathit{u1}_{1} + \left(-\frac{2}{5}\right) \mathit{u0}_{1} \mathit{u2}_{2} + \frac{1}{5} \mathit{u0}_{1} \mathit{u3}_{3} + \frac{16}{25} \mathit{u1}_{5} \mathit{u3}_{0} + \frac{2}{5} \mathit{u1}_{4} \mathit{u2}_{0} + \frac{6}{5} \mathit{u1}_{4} \mathit{u3}_{1} + \frac{2}{5} \mathit{u1}_{3} \mathit{u1}_{0} + \frac{12}{5} \mathit{u1}_{3} \mathit{u3}_{2} + \frac{12}{5} \mathit{u1}_{2} \mathit{u3}_{3} + \frac{6}{5} \mathit{u1}_{1} \mathit{u3}_{4} + \left(-\frac{1}{5}\right) \mathit{u1}_{0} \mathit{u2}_{4} + \frac{1}{5} \mathit{u1}_{0} \mathit{u3}_{5} + \left(-\frac{8}{25}\right) \mathit{u2}_{6} \mathit{u3}_{0} + \frac{1}{25} \mathit{u2}_{5} \mathit{u2}_{0} + \left(-\frac{21}{25}\right) \mathit{u2}_{5} \mathit{u3}_{1} + \frac{6}{5

...

P_9 = \frac{36}{625} \mathit{u2}_{0} \mathit{u3}_{0}^{3} z_{0} + \left(-\frac{36}{625}\right) \mathit{u3}_{1} \mathit{u3}_{0}^{3} z_{0} + \frac{9}{625} \mathit{u3}_{0}^{4} z_{1} + \left(-\frac{18}{125}\right) \mathit{u0}_{0} \mathit{u3}_{0}^{2} z_{0} + \left(-\frac{36}{125}\right) \mathit{u1}_{0} \mathit{u2}_{0} \mathit{u3}_{0} z_{0} + \frac{36}{125} \mathit{u1}_{0} \mathit{u3}_{1} \mathit{u3}_{0} z_{0} + \left(-\frac{18}{125}\right) \mathit{u1}_{0} \mathit{u3}_{0}^{2} z_{1} + \frac{18}{125} \mathit{u2}_{2} \mathit{u3}_{0}^{2} z_{0} + \frac{18}{125} \mathit{u2}_{1} \mathit{u2}_{0} \mathit{u3}_{0} z_{0} + \frac{36}{125} \mathit{u2}_{1} \mathit{u3}_{1} \mathit{u3}_{0} z_{0} + \left(-\frac{6}{125}\right) \mathit{u2}_{0}^{3} z_{0} + \frac{18}{125} \mathit{u2}_{0}^{2} \mathit{u3}_{1} z_{0} + \left(-\frac{18}{125}\right) \mathit{u2}_{0}^{2} \mathit{u3}_{0} z_{1} + \frac{36}{125} \mathit{u2}_{0} \mathit{u3}_{2} \mathit{u3}_{0} z_{0} + \frac{3}{25} \mathit{u2}_{0} \mathit{u3}_{1}^{2} z_{0} + \frac{18}{125} \mathit{u2}_{0} \mathit{u3}_{1} \mathit{u3}_{0} z_{1} + \left(-\frac{18}{125}\right) \mathit{u2}_{0} \mathit{u3}_{0}^{2} z_{2} + \left(-\frac{18}{125}\right) \mathit{u3}_{3} \mathit{u3}_{0}^{2} z_{0} + \left(-\frac{18}{25}\right) \mathit{u3}_{2} \mathit{u3}_{1} \mathit{u3}_{0} z_{0} + \frac{18}{125} \mathit{u3}_{2} \mathit{u3}_{0}^{2} z_{1} + \left(-\frac{27}{125}\right) \mathit{u3}_{1}^{3} z_{0} + \frac{27}{125} \mathit{u3}_{1}^{2} \mathit{u3}_{0} z_{1} + \left(-\frac{6}{125}\right) \mathit{u3}_{0}^{3} z_{3} + \frac{18}{25} \mathit{u0}_{2} \mathit{u3}_{0} z_{0} + \frac{18}{25} \mathit{u0}_{1} \mathit{u2}_{0} z_{0} + \left(-\frac{12}{25}\right) \mathit{u0}_{1} \mathit{u3}_{1} z_{0} + \frac{36}{25} \mathit{u0}_{1} \mathit{u3}_{0} z_{1} + \frac{36}{25} \mathit{u0}_{0} \mathit{u1}_{0} z_{0} + \left(-\frac{18}{25}\right) \mathit{u0}_{0} \mathit{u2}_{1} z_{0} + \frac{36}{25} \mathit{u0}_{0} \mathit{u2}_{0} z_{1} + \left(-\frac{18}{25}\right) \mathit{u0}_{0} \mathit{u3}_{1} z_{1} + \frac{36}{25} \mathit{u0}_{0} \mathit{u3}_{0} z_{2} + \left(-\frac{6}{25}\right) \mathit{u1}_{2} \mathit{u2}_{0} z_{0} + \frac{6}{25} \mathit{u1}_{2} \mathit{u3}_{1} z_{0} + \frac{18}{25} \mathit{u1}_{2} \mathit{u3}_{0} z_{1} + \left(-\frac{12}{25}\right) \mathit{u1}_{1} \mathit{u2}_{1} z_{0} + \frac{18}{25} \mathit{u1}_{1} \mathit{u2}_{0} z_{1} + \frac{12}{25} \mathit{u1}_{1} \mathit{u3}_{2} z_{0} + \left(-\frac{18}{25}\right) \mathit{u1}_{1} \mathit{u3}_{1} z_{1} + \frac{36}{25} \mathit{u1}_{1} \mathit{u3}_{0} z_{2} + \frac{18}{25} \mathit{u1}_{0}^{2} z_{1} + \left(-\frac{12}{25}\right) \mathit{u1}_{0} \mathit{u2}_{2} z_{0} + \frac{36}{25} \mathit{u1}_{0} \mathit{u2}_{0} z_{2} + \frac{12}{25} \mathit{u1}_{0} \mathit{u3}_{3} z_{0} + \left(-\frac{12}{25}\right) \mathit{u1}_{0} \mathit{u3}_{2} z_{1} + \frac{36}{25} \mathit{u1}_{0} \mathit{u3}_{0} z_{3} + \frac{21}{125} \mathit{u2}_{4} \mathit{u3}_{0} z_{0} + \frac{3}{25} \mathit{u2}_{3} \mathit{u2}_{0} z_{0} + \frac{51}{125} \mathit{u2}_{3} \mathit{u3}_{1} z_{0} + \frac{12}{25} \mathit{u2}_{2} \mathit{u2}_{1} z_{0} + \left(-\frac{6}{25}\right) \mathit{u2}_{2} \mathit{u2}_{0} z_{1} + \frac{54}{125} \mathit{u2}_{2} \mathit{u3}_{2} z_{0} + \frac{3}{25} \mathit{u2}_{2} \mathit{u3}_{1} z_{1} + \frac{18}{25} \mathit{u2}_{2} \mathit{u3}_{0} z_{2} + \left(-\frac{9}{25}\right) \mathit{u2}_{1}^{2} z_{1} + \frac{18}{25} \mathit{u2}_{1} \mathit{u2}_{0} z_{2} + \frac{36}{125} \mathit{u2}_{1} \mathit{u3}_{3} z_{0} + \frac{9}{25} \mathit{u2}_{1} \mathit{u3}_{2} z_{1} + \left(-\frac{24}{25}\right) \mathit{u2}_{1} \mathit{u3}_{1} z_{2} + \frac{36}{25} \mathit{u2}_{1} \mathit{u3}_{0} z_{3} + \frac{18}{25} \mathit{u2}_{0}^{2} z_{3} + \frac{27}{125} \mathit{u2}_{0} \mathit{u3}_{4} z_{0} + \frac{3}{25} \mathit{u2}_{0} \mathit{u3}_{3} z_{1} + \left(-\frac{6}{25}\right) \mathit{u2}_{0} \mathit{u3}_{2} z_{2} + \frac{18}{25} \mathit{u2}_{0} \mathit{u3}_{1} z_{3} + \frac{36}{25} \mathit{u2}_{0} \mathit{u3}_{0} z_{4} + \left(-\frac{21}{12

\left\{\begin{array}{rcl}f_{1,0} & = & \left(-\frac{9}{625}\right) \mathit{u0}_{1} \mathit{u3}_{0}^{4} + \frac{36}{625} \mathit{u1}_{0} \mathit{u2}_{1} \mathit{u3}_{0}^{3} + \frac{108}{625} \mathit{u1}_{0} \mathit{u2}_{0} \mathit{u3}_{1} \mathit{u3}_{0}^{2} + \left(-\frac{36}{625}\right) \mathit{u1}_{0} \mathit{u3}_{2} \mathit{u3}_{0}^{3} + \left(-\frac{108}{625}\right) \mathit{u1}_{0} \mathit{u3}_{1}^{2} \mathit{u3}_{0}^{2} + \frac{36}{625} \mathit{u2}_{3} \mathit{u3}_{0}^{4} + \frac{36}{625} \mathit{u2}_{2} \mathit{u2}_{0} \mathit{u3}_{0}^{3} + \frac{324}{625} \mathit{u2}_{2} \mathit{u3}_{1} \mathit{u3}_{0}^{3} + \frac{216}{625} \mathit{u2}_{1} \mathit{u2}_{0} \mathit{u3}_{1} \mathit{u3}_{0}^{2} + \frac{324}{625} \mathit{u2}_{1} \mathit{u3}_{2} \mathit{u3}_{0}^{3} + \frac{648}{625} \mathit{u2}_{1} \mathit{u3}_{1}^{2} \mathit{u3}_{0}^{2} + \frac{108}{625} \mathit{u2}_{0}^{2} \mathit{u3}_{2} \mathit{u3}_{0}^{2} + \frac{216}{625} \mathit{u2}_{0}^{2} \mathit{u3}_{1}^{2} \mathit{u3}_{0} + \frac{72}{625} \mathit{u2}_{0} \mathit{u3}_{3} \mathit{u3}_{0}^{3} + \frac{324}{625} \mathit{u2}_{0} \mathit{u3}_{2} \mathit{u3}_{1} \mathit{u3}_{0}^{2} + \left(-\frac{36}{625}\right) \mathit{u3}_{4} \mathit{u3}_{0}^{4} + \left(-\frac{432}{625}\right) \mathit{u3}_{3} \mathit{u3}_{1} \mathit{u3}_{0}^{3} + \left(-\frac{324}{625}\right) \mathit{u3}_{2}^{2} \mathit{u3}_{0}^{3} + \left(-\frac{1296}{625}\right) \mathit{u3}_{2} \mathit{u3}_{1}^{2} \mathit{u3}_{0}^{2} + \left(-\frac{216}{625}\right) \mathit{u3}_{1}^{4} \mathit{u3}_{0} + \left(-\frac{12}{125}\right) \mathit{u0}_{3} \mathit{u3}_{0}^{3} + \left(-\frac{108}{125}\right) \mathit{u0}_{2} \mathit{u3}_{1} \mathit{u3}_{0}^{2} + \frac{18}{125} \mathit{u0}_{1} \mathit{u2}_{0}^{2} \mathit{u3}_{0} + \left(-\frac{18}{25}\right) \mathit{u0}_{1} \mathit{u2}_{0} \mathit{u3}_{1} \mathit{u3}_{0} + \left(-\frac{126}{125}\right) \mathit{u0}_{1} \mathit{u3}_{2} \mathit{u3}_{0}^{2} + \left(-\frac{27}{25}\right) \mathit{u0}_{1} \mathit{u3}_{1}^{2} \mathit{u3}_{0} + \left(-\frac{36}{125}\right) \mathit{u0}_{0} \mathit{u1}_{0} \mathit{u3}_{1} \mathit{u3}_{0} + \left(-\frac{36}{125}\right) \mathit{u0}_{0} \mathit{u2}_{0} \mathit{u3}_{2} \mathit{u3}_{0} + \left(-\frac{36}{125}\right) \mathit{u0}_{0} \mathit{u2}_{0} \mathit{u3}_{1}^{2} + \left(-\frac{36}{125}\right) \mathit{u0}_{0} \mathit{u3}_{3} \mathit{u3}_{0}^{2} + \left(-\frac{108}{125}\right) \mathit{u0}_{0} \mathit{u3}_{2} \mathit{u3}_{1} \mathit{u3}_{0} + \left(-\frac{36}{125}\right) \mathit{u1}_{3} \mathit{u2}_{0} \mathit{u3}_{0}^{2} + \frac{36}{125} \mathit{u1}_{3} \mathit{u3}_{1} \mathit{u3}_{0}^{2} + \left(-\frac{108}{125}\right) \mathit{u1}_{2} \mathit{u2}_{1} \mathit{u3}_{0}^{2} + \left(-\frac{36}{125}\right) \mathit{u1}_{2} \mathit{u2}_{0}^{2} \mathit{u3}_{0} + \left(-\frac{72}{125}\right) \mathit{u1}_{2} \mathit{u2}_{0} \mathit{u3}_{1} \mathit{u3}_{0} + \frac{108}{125} \mathit{u1}_{2} \mathit{u3}_{2} \mathit{u3}_{0}^{2} + \frac{108}{125} \mathit{u1}_{2} \mathit{u3}_{1}^{2} \mathit{u3}_{0} + \left(-\frac{36}{125}\right) \mathit{u1}_{1} \mathit{u1}_{0} \mathit{u2}_{0} \mathit{u3}_{0} + \frac{36}{125} \mathit{u1}_{1} \mathit{u1}_{0} \mathit{u3}_{1} \mathit{u3}_{0} + \left(-\frac{108}{125}\right) \mathit{u1}_{1} \mathit{u2}_{2} \mathit{u3}_{0}^{2} + \left(-\frac{72}{125}\right) \mathit{u1}_{1} \mathit{u2}_{1} \mathit{u2}_{0} \mathit{u3}_{0} + \left(-\frac{216}{125}\right) \mathit{u1}_{1} \mathit{u2}_{1} \mathit{u3}_{1} \mathit{u3}_{0} + \left(-\frac{72}{125}\right) \mathit{u1}_{1} \mathit{u2}_{0}^{2} \mathit{u3}_{1} + \left(-\frac{36}{125}\right) \mathit{u1}_{1} \mathit{u2}_{0} \mathit{u3}_{2} \mathit{u3}_{0} + \frac{72}{125} \mathit{u1}_{1} \mathit{u2}_{0} \mathit{u3}_{1}^{2} + \frac{108}{125} \mathit{u1}_{1} \mathit{u3}_{3} \mathit{u3}_{0}^{2} + \frac{324}{125} \mathit{u1}_{1} \mathit{u3}_{2} \mathit{u3}_{1} \mathit{u3}_{0} + \left(-\frac{36}{125}\right) \mathit{u1}_{0}^{2} \mathit{u2}_{1} \mathit{u3}_{0} + \left(-\frac{36}{125}\right) \mathit{u1}_{0}^

Number of ansatz constants: 185
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [3, 0, 0, 0, 0, 2, 0, 1, 2, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 5, 0, 0, 1, 0, 0, 0, 0, 0, 0], [5, 3, 0, 0, 0, 10, 3, 1, 0, 1, 1, 0, 0, 0, 2, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 2], [1, 10, 0, 1, 5, 3, 1, 2, 0, 10, 1, 0, 0, 2, 3, 2], [0, 10, 2, 3, 2, 10, 1, 5, 3, 0, 5], [2, 10, 10, 3, 2, 5], [5, 1, 10, 2], [5, 2], [2]]
Generator for the z's in Ring of operator polynomials in (u0, u1, u2, u3, z) over Differential Ring [[Rational Field], (0,)]
-------------------------------------------------------------------------------------------------------------------
Computed the almost commutator of order 10


P_10 = 1 \mathit{u0}_{3} \mathit{u3}_{0} z_{0} + 1 \mathit{u0}_{2} \mathit{u2}_{0} z_{0} + 3 \mathit{u0}_{2} \mathit{u3}_{0} z_{1} + 1 \mathit{u0}_{1} \mathit{u1}_{0} z_{0} + 2 \mathit{u0}_{1} \mathit{u2}_{0} z_{1} + 3 \mathit{u0}_{1} \mathit{u3}_{0} z_{2} + 1 \mathit{u0}_{0}^{2} z_{0} + 2 \mathit{u0}_{0} \mathit{u1}_{0} z_{1} + 2 \mathit{u0}_{0} \mathit{u2}_{0} z_{2} + 2 \mathit{u0}_{0} \mathit{u3}_{0} z_{3} + 1 \mathit{u1}_{3} \mathit{u3}_{0} z_{1} + 1 \mathit{u1}_{2} \mathit{u2}_{0} z_{1} + 3 \mathit{u1}_{2} \mathit{u3}_{0} z_{2} + 1 \mathit{u1}_{1} \mathit{u1}_{0} z_{1} + 2 \mathit{u1}_{1} \mathit{u2}_{0} z_{2} + 3 \mathit{u1}_{1} \mathit{u3}_{0} z_{3} + 1 \mathit{u1}_{0}^{2} z_{2} + 1 \mathit{u1}_{0} \mathit{u2}_{1} z_{2} + 2 \mathit{u1}_{0} \mathit{u2}_{0} z_{3} + 1 \mathit{u1}_{0} \mathit{u3}_{1} z_{3} + 2 \mathit{u1}_{0} \mathit{u3}_{0} z_{4} + 1 \mathit{u2}_{3} \mathit{u3}_{0} z_{2} + 1 \mathit{u2}_{2} \mathit{u2}_{0} z_{2} + 3 \mathit{u2}_{2} \mathit{u3}_{0} z_{3} + 2 \mathit{u2}_{1} \mathit{u2}_{0} z_{3} + 3 \mathit{u2}_{1} \mathit{u3}_{0} z_{4} + 1 \mathit{u2}_{0}^{2} z_{4} + 1 \mathit{u2}_{0} \mathit{u3}_{2} z_{3} + 2 \mathit{u2}_{0} \mathit{u3}_{1} z_{4} + 2 \mathit{u2}_{0} \mathit{u3}_{0} z_{5} + 1 \mathit{u3}_{3} \mathit{u3}_{0} z_{3} + 3 \mathit{u3}_{2} \mathit{u3}_{0} z_{4} + 3 \mathit{u3}_{1} \mathit{u3}_{0} z_{5} + 1 \mathit{u3}_{0}^{2} z_{6} + 1 \mathit{u0}_{5} z_{0} + 5 \mathit{u0}_{4} z_{1} + 10 \mathit{u0}_{3} z_{2} + 10 \mathit{u0}_{2} z_{3} + 5 \mathit{u0}_{1} z_{4} + 2 \mathit{u0}_{0} z_{5} + 1 \mathit{u1}_{5} z_{1} + 5 \mathit{u1}_{4} z_{2} + 10 \mathit{u1}_{3} z_{3} + 10 \mathit{u1}_{2} z_{4} + 5 \mathit{u1}_{1} z_{5} + 2 \mathit{u1}_{0} z_{6} + 1 \mathit{u2}_{5} z_{2} + 5 \mathit{u2}_{4} z_{3} + 10 \mathit{u2}_{3} z_{4} + 10 \mathit{u2}_{2} z_{5} + 5 \mathit{u2}_{1} z_{6} + 2 \mathit{u2}_{0} z_{7} + 1 \mathit{u3}_{5} z_{3} + 5 \mathit{u3}_{4} z_{4} + 10 \mathit{u3}_{3} z_{5} + 10 \mathit{u3}_{2} z_{6} + 5 \mathit{u3}_{1} z_{7} + 2 \mathit{u3}_{0} z_{8} + 1 z_{10}

... This commutes with L
CPU times: user 39.6 s, sys: 159 ms, total: 39.8 s
Wall time: 39.8 s


In [114]:
%time seq=get_sequence(5, 10, quasi_commutting_generic_diff)

-------------------------------------------------------------------------------------------------------------------
Computed the almost commutator of order 5


P_5 = 1 \mathit{u0}_{0} z_{0} + 1 \mathit{u1}_{0} z_{1} + 1 \mathit{u2}_{0} z_{2} + 1 \mathit{u3}_{0} z_{3} + 1 z_{5}

... This commutes with L
-------------------------------------------------------------------------------------------------------------------
Computed the almost commutator of order 6


P_6 = \left(-\frac{4}{125}\right) \mathit{u3}_{0}^{3} z_{0} + \frac{6}{25} \mathit{u1}_{0} \mathit{u3}_{0} z_{0} + \left(-\frac{3}{25}\right) \mathit{u2}_{1} \mathit{u3}_{0} z_{0} + \frac{3}{25} \mathit{u2}_{0}^{2} z_{0} + \left(-\frac{6}{25}\right) \mathit{u2}_{0} \mathit{u3}_{1} z_{0} + \frac{6}{25} \mathit{u2}_{0} \mathit{u3}_{0} z_{1} + \left(-\frac{3}{25}\right) \mathit{u3}_{2} \mathit{u3}_{0} z_{0} + \left(-\frac{1}{25}\right) \mathit{u3}_{1}^{2} z_{0} + \left(-\frac{3}{25}\right) \mathit{u3}_{1} \mathit{u3}_{0} z_{1} + \frac{3}{25} \mathit{u3}_{0}^{2} z_{2} + \frac{3}{5} \mathit{u0}_{1} z_{0} + \frac{6}{5} \mathit{u0}_{0} z_{1} + \frac{2}{5} \mathit{u1}_{2} z_{0} + \frac{3}{5} \mathit{u1}_{1} z_{1} + \frac{6}{5} \mathit{u1}_{0} z_{2} + \left(-\frac{1}{5}\right) \mathit{u2}_{3} z_{0} + \frac{2}{5} \mathit{u2}_{2} z_{1} + \frac{3}{5} \mathit{u2}_{1} z_{2} + \frac{6}{5} \mathit{u2}_{0} z_{3} + \left(-\frac{1}{25}\right) \mathit{u3}_{4} z_{0} + \left(-\frac{1}{5}\right) \mathit{u3}_{3} z_{1} + \frac{2}{5} \mathit{u3}_{2} z_{2} + \frac{3}{5} \mathit{u3}_{1} z_{3} + \frac{6}{5} \mathit{u3}_{0} z_{4} + 1 z_{6}

\left\{\begin{array}{rcl}f_{1,0} & = & \left(-\frac{12}{125}\right) \mathit{u1}_{0} \mathit{u3}_{1} \mathit{u3}_{0}^{2} + \left(-\frac{12}{125}\right) \mathit{u2}_{0} \mathit{u3}_{2} \mathit{u3}_{0}^{2} + \left(-\frac{24}{125}\right) \mathit{u2}_{0} \mathit{u3}_{1}^{2} \mathit{u3}_{0} + \left(-\frac{12}{125}\right) \mathit{u3}_{3} \mathit{u3}_{0}^{3} + \left(-\frac{72}{125}\right) \mathit{u3}_{2} \mathit{u3}_{1} \mathit{u3}_{0}^{2} + \left(-\frac{24}{125}\right) \mathit{u3}_{1}^{3} \mathit{u3}_{0} + \left(-\frac{3}{25}\right) \mathit{u0}_{2} \mathit{u3}_{0}^{2} + \left(-\frac{6}{25}\right) \mathit{u0}_{1} \mathit{u2}_{0} \mathit{u3}_{0} + \frac{3}{25} \mathit{u0}_{1} \mathit{u3}_{1} \mathit{u3}_{0} + \frac{6}{25} \mathit{u1}_{3} \mathit{u3}_{0}^{2} + \frac{6}{25} \mathit{u1}_{2} \mathit{u2}_{0} \mathit{u3}_{0} + \frac{18}{25} \mathit{u1}_{2} \mathit{u3}_{1} \mathit{u3}_{0} + \frac{6}{25} \mathit{u1}_{1} \mathit{u1}_{0} \mathit{u3}_{0} + \frac{12}{25} \mathit{u1}_{1} \mathit{u2}_{0} \mathit{u3}_{1} + \frac{18}{25} \mathit{u1}_{1} \mathit{u3}_{2} \mathit{u3}_{0} + \frac{6}{25} \mathit{u1}_{0}^{2} \mathit{u3}_{1} + \left(-\frac{3}{25}\right) \mathit{u1}_{0} \mathit{u2}_{2} \mathit{u3}_{0} + \frac{6}{25} \mathit{u1}_{0} \mathit{u2}_{1} \mathit{u2}_{0} + \left(-\frac{9}{25}\right) \mathit{u1}_{0} \mathit{u2}_{1} \mathit{u3}_{1} + \frac{3}{25} \mathit{u1}_{0} \mathit{u3}_{3} \mathit{u3}_{0} + \left(-\frac{1}{5}\right) \mathit{u1}_{0} \mathit{u3}_{2} \mathit{u3}_{1} + \left(-\frac{3}{25}\right) \mathit{u2}_{4} \mathit{u3}_{0}^{2} + \frac{3}{25} \mathit{u2}_{3} \mathit{u2}_{0} \mathit{u3}_{0} + \left(-\frac{3}{5}\right) \mathit{u2}_{3} \mathit{u3}_{1} \mathit{u3}_{0} + \frac{18}{25} \mathit{u2}_{2} \mathit{u2}_{1} \mathit{u3}_{0} + \frac{6}{25} \mathit{u2}_{2} \mathit{u2}_{0}^{2} + \left(-\frac{12}{25}\right) \mathit{u2}_{2} \mathit{u2}_{0} \mathit{u3}_{1} + \left(-\frac{27}{25}\right) \mathit{u2}_{2} \mathit{u3}_{2} \mathit{u3}_{0} + \frac{6}{25} \mathit{u2}_{1}^{2} \mathit{u2}_{0} + \left(-\frac{3}{5}\right) \mathit{u2}_{1} \mathit{u2}_{0} \mathit{u3}_{2} + \left(-\frac{21}{25}\right) \mathit{u2}_{1} \mathit{u3}_{3} \mathit{u3}_{0} + \left(-\frac{6}{25}\right) \mathit{u2}_{0}^{2} \mathit{u3}_{3} + \left(-\frac{9}{25}\right) \mathit{u2}_{0} \mathit{u3}_{4} \mathit{u3}_{0} + \left(-\frac{8}{25}\right) \mathit{u2}_{0} \mathit{u3}_{3} \mathit{u3}_{1} + \left(-\frac{1}{5}\right) \mathit{u2}_{0} \mathit{u3}_{2}^{2} + \left(-\frac{27}{125}\right) \mathit{u3}_{5} \mathit{u3}_{0}^{2} + \left(-\frac{7}{5}\right) \mathit{u3}_{4} \mathit{u3}_{1} \mathit{u3}_{0} + \left(-\frac{66}{25}\right) \mathit{u3}_{3} \mathit{u3}_{2} \mathit{u3}_{0} + \left(-\frac{48}{25}\right) \mathit{u3}_{3} \mathit{u3}_{1}^{2} + \left(-\frac{72}{25}\right) \mathit{u3}_{2}^{2} \mathit{u3}_{1} + \left(-\frac{3}{5}\right) \mathit{u0}_{4} \mathit{u3}_{0} + \left(-\frac{3}{5}\right) \mathit{u0}_{3} \mathit{u2}_{0} + \left(-\frac{3}{5}\right) \mathit{u0}_{3} \mathit{u3}_{1} + \left(-\frac{3}{5}\right) \mathit{u0}_{2} \mathit{u1}_{0} + \left(-\frac{3}{5}\right) \mathit{u0}_{2} \mathit{u2}_{1} + \left(-\frac{2}{5}\right) \mathit{u0}_{2} \mathit{u3}_{2} + \left(-\frac{6}{5}\right) \mathit{u0}_{1} \mathit{u0}_{0} + \left(-\frac{3}{5}\right) \mathit{u0}_{1} \mathit{u1}_{1} + \left(-\frac{2}{5}\right) \mathit{u0}_{1} \mathit{u2}_{2} + \frac{1}{5} \mathit{u0}_{1} \mathit{u3}_{3} + \frac{16}{25} \mathit{u1}_{5} \mathit{u3}_{0} + \frac{2}{5} \mathit{u1}_{4} \mathit{u2}_{0} + \frac{6}{5} \mathit{u1}_{4} \mathit{u3}_{1} + \frac{2}{5} \mathit{u1}_{3} \mathit{u1}_{0} + \frac{12}{5} \mathit{u1}_{3} \mathit{u3}_{2} + \frac{12}{5} \mathit{u1}_{2} \mathit{u3}_{3} + \frac{6}{5} \mathit{u1}_{1} \mathit{u3}_{4} + \left(-\frac{1}{5}\right) \mathit{u1}_{0} \mathit{u2}_{4} + \frac{1}{5} \mathit{u1}_{0} \mathit{u3}_{5} + \left(-\frac{8}{25}\right) \mathit{u2}_{6} \mathit{u3}_{0} + \frac{1}{25} \mathit{u2}_{5} \mathit{u2}_{0} + \left(-\frac{21}{25}\right) \mathit{u2}_{5} \mathit{u3}_{1} + \frac{6}{5

...

P_9 = \frac{36}{625} \mathit{u2}_{0} \mathit{u3}_{0}^{3} z_{0} + \left(-\frac{36}{625}\right) \mathit{u3}_{1} \mathit{u3}_{0}^{3} z_{0} + \frac{9}{625} \mathit{u3}_{0}^{4} z_{1} + \left(-\frac{18}{125}\right) \mathit{u0}_{0} \mathit{u3}_{0}^{2} z_{0} + \left(-\frac{36}{125}\right) \mathit{u1}_{0} \mathit{u2}_{0} \mathit{u3}_{0} z_{0} + \frac{36}{125} \mathit{u1}_{0} \mathit{u3}_{1} \mathit{u3}_{0} z_{0} + \left(-\frac{18}{125}\right) \mathit{u1}_{0} \mathit{u3}_{0}^{2} z_{1} + \frac{18}{125} \mathit{u2}_{2} \mathit{u3}_{0}^{2} z_{0} + \frac{18}{125} \mathit{u2}_{1} \mathit{u2}_{0} \mathit{u3}_{0} z_{0} + \frac{36}{125} \mathit{u2}_{1} \mathit{u3}_{1} \mathit{u3}_{0} z_{0} + \left(-\frac{6}{125}\right) \mathit{u2}_{0}^{3} z_{0} + \frac{18}{125} \mathit{u2}_{0}^{2} \mathit{u3}_{1} z_{0} + \left(-\frac{18}{125}\right) \mathit{u2}_{0}^{2} \mathit{u3}_{0} z_{1} + \frac{36}{125} \mathit{u2}_{0} \mathit{u3}_{2} \mathit{u3}_{0} z_{0} + \frac{3}{25} \mathit{u2}_{0} \mathit{u3}_{1}^{2} z_{0} + \frac{18}{125} \mathit{u2}_{0} \mathit{u3}_{1} \mathit{u3}_{0} z_{1} + \left(-\frac{18}{125}\right) \mathit{u2}_{0} \mathit{u3}_{0}^{2} z_{2} + \left(-\frac{18}{125}\right) \mathit{u3}_{3} \mathit{u3}_{0}^{2} z_{0} + \left(-\frac{18}{25}\right) \mathit{u3}_{2} \mathit{u3}_{1} \mathit{u3}_{0} z_{0} + \frac{18}{125} \mathit{u3}_{2} \mathit{u3}_{0}^{2} z_{1} + \left(-\frac{27}{125}\right) \mathit{u3}_{1}^{3} z_{0} + \frac{27}{125} \mathit{u3}_{1}^{2} \mathit{u3}_{0} z_{1} + \left(-\frac{6}{125}\right) \mathit{u3}_{0}^{3} z_{3} + \frac{18}{25} \mathit{u0}_{2} \mathit{u3}_{0} z_{0} + \frac{18}{25} \mathit{u0}_{1} \mathit{u2}_{0} z_{0} + \left(-\frac{12}{25}\right) \mathit{u0}_{1} \mathit{u3}_{1} z_{0} + \frac{36}{25} \mathit{u0}_{1} \mathit{u3}_{0} z_{1} + \frac{36}{25} \mathit{u0}_{0} \mathit{u1}_{0} z_{0} + \left(-\frac{18}{25}\right) \mathit{u0}_{0} \mathit{u2}_{1} z_{0} + \frac{36}{25} \mathit{u0}_{0} \mathit{u2}_{0} z_{1} + \left(-\frac{18}{25}\right) \mathit{u0}_{0} \mathit{u3}_{1} z_{1} + \frac{36}{25} \mathit{u0}_{0} \mathit{u3}_{0} z_{2} + \left(-\frac{6}{25}\right) \mathit{u1}_{2} \mathit{u2}_{0} z_{0} + \frac{6}{25} \mathit{u1}_{2} \mathit{u3}_{1} z_{0} + \frac{18}{25} \mathit{u1}_{2} \mathit{u3}_{0} z_{1} + \left(-\frac{12}{25}\right) \mathit{u1}_{1} \mathit{u2}_{1} z_{0} + \frac{18}{25} \mathit{u1}_{1} \mathit{u2}_{0} z_{1} + \frac{12}{25} \mathit{u1}_{1} \mathit{u3}_{2} z_{0} + \left(-\frac{18}{25}\right) \mathit{u1}_{1} \mathit{u3}_{1} z_{1} + \frac{36}{25} \mathit{u1}_{1} \mathit{u3}_{0} z_{2} + \frac{18}{25} \mathit{u1}_{0}^{2} z_{1} + \left(-\frac{12}{25}\right) \mathit{u1}_{0} \mathit{u2}_{2} z_{0} + \frac{36}{25} \mathit{u1}_{0} \mathit{u2}_{0} z_{2} + \frac{12}{25} \mathit{u1}_{0} \mathit{u3}_{3} z_{0} + \left(-\frac{12}{25}\right) \mathit{u1}_{0} \mathit{u3}_{2} z_{1} + \frac{36}{25} \mathit{u1}_{0} \mathit{u3}_{0} z_{3} + \frac{21}{125} \mathit{u2}_{4} \mathit{u3}_{0} z_{0} + \frac{3}{25} \mathit{u2}_{3} \mathit{u2}_{0} z_{0} + \frac{51}{125} \mathit{u2}_{3} \mathit{u3}_{1} z_{0} + \frac{12}{25} \mathit{u2}_{2} \mathit{u2}_{1} z_{0} + \left(-\frac{6}{25}\right) \mathit{u2}_{2} \mathit{u2}_{0} z_{1} + \frac{54}{125} \mathit{u2}_{2} \mathit{u3}_{2} z_{0} + \frac{3}{25} \mathit{u2}_{2} \mathit{u3}_{1} z_{1} + \frac{18}{25} \mathit{u2}_{2} \mathit{u3}_{0} z_{2} + \left(-\frac{9}{25}\right) \mathit{u2}_{1}^{2} z_{1} + \frac{18}{25} \mathit{u2}_{1} \mathit{u2}_{0} z_{2} + \frac{36}{125} \mathit{u2}_{1} \mathit{u3}_{3} z_{0} + \frac{9}{25} \mathit{u2}_{1} \mathit{u3}_{2} z_{1} + \left(-\frac{24}{25}\right) \mathit{u2}_{1} \mathit{u3}_{1} z_{2} + \frac{36}{25} \mathit{u2}_{1} \mathit{u3}_{0} z_{3} + \frac{18}{25} \mathit{u2}_{0}^{2} z_{3} + \frac{27}{125} \mathit{u2}_{0} \mathit{u3}_{4} z_{0} + \frac{3}{25} \mathit{u2}_{0} \mathit{u3}_{3} z_{1} + \left(-\frac{6}{25}\right) \mathit{u2}_{0} \mathit{u3}_{2} z_{2} + \frac{18}{25} \mathit{u2}_{0} \mathit{u3}_{1} z_{3} + \frac{36}{25} \mathit{u2}_{0} \mathit{u3}_{0} z_{4} + \left(-\frac{21}{12

\left\{\begin{array}{rcl}f_{1,0} & = & \left(-\frac{9}{625}\right) \mathit{u0}_{1} \mathit{u3}_{0}^{4} + \frac{36}{625} \mathit{u1}_{0} \mathit{u2}_{1} \mathit{u3}_{0}^{3} + \frac{108}{625} \mathit{u1}_{0} \mathit{u2}_{0} \mathit{u3}_{1} \mathit{u3}_{0}^{2} + \left(-\frac{36}{625}\right) \mathit{u1}_{0} \mathit{u3}_{2} \mathit{u3}_{0}^{3} + \left(-\frac{108}{625}\right) \mathit{u1}_{0} \mathit{u3}_{1}^{2} \mathit{u3}_{0}^{2} + \frac{36}{625} \mathit{u2}_{3} \mathit{u3}_{0}^{4} + \frac{36}{625} \mathit{u2}_{2} \mathit{u2}_{0} \mathit{u3}_{0}^{3} + \frac{324}{625} \mathit{u2}_{2} \mathit{u3}_{1} \mathit{u3}_{0}^{3} + \frac{216}{625} \mathit{u2}_{1} \mathit{u2}_{0} \mathit{u3}_{1} \mathit{u3}_{0}^{2} + \frac{324}{625} \mathit{u2}_{1} \mathit{u3}_{2} \mathit{u3}_{0}^{3} + \frac{648}{625} \mathit{u2}_{1} \mathit{u3}_{1}^{2} \mathit{u3}_{0}^{2} + \frac{108}{625} \mathit{u2}_{0}^{2} \mathit{u3}_{2} \mathit{u3}_{0}^{2} + \frac{216}{625} \mathit{u2}_{0}^{2} \mathit{u3}_{1}^{2} \mathit{u3}_{0} + \frac{72}{625} \mathit{u2}_{0} \mathit{u3}_{3} \mathit{u3}_{0}^{3} + \frac{324}{625} \mathit{u2}_{0} \mathit{u3}_{2} \mathit{u3}_{1} \mathit{u3}_{0}^{2} + \left(-\frac{36}{625}\right) \mathit{u3}_{4} \mathit{u3}_{0}^{4} + \left(-\frac{432}{625}\right) \mathit{u3}_{3} \mathit{u3}_{1} \mathit{u3}_{0}^{3} + \left(-\frac{324}{625}\right) \mathit{u3}_{2}^{2} \mathit{u3}_{0}^{3} + \left(-\frac{1296}{625}\right) \mathit{u3}_{2} \mathit{u3}_{1}^{2} \mathit{u3}_{0}^{2} + \left(-\frac{216}{625}\right) \mathit{u3}_{1}^{4} \mathit{u3}_{0} + \left(-\frac{12}{125}\right) \mathit{u0}_{3} \mathit{u3}_{0}^{3} + \left(-\frac{108}{125}\right) \mathit{u0}_{2} \mathit{u3}_{1} \mathit{u3}_{0}^{2} + \frac{18}{125} \mathit{u0}_{1} \mathit{u2}_{0}^{2} \mathit{u3}_{0} + \left(-\frac{18}{25}\right) \mathit{u0}_{1} \mathit{u2}_{0} \mathit{u3}_{1} \mathit{u3}_{0} + \left(-\frac{126}{125}\right) \mathit{u0}_{1} \mathit{u3}_{2} \mathit{u3}_{0}^{2} + \left(-\frac{27}{25}\right) \mathit{u0}_{1} \mathit{u3}_{1}^{2} \mathit{u3}_{0} + \left(-\frac{36}{125}\right) \mathit{u0}_{0} \mathit{u1}_{0} \mathit{u3}_{1} \mathit{u3}_{0} + \left(-\frac{36}{125}\right) \mathit{u0}_{0} \mathit{u2}_{0} \mathit{u3}_{2} \mathit{u3}_{0} + \left(-\frac{36}{125}\right) \mathit{u0}_{0} \mathit{u2}_{0} \mathit{u3}_{1}^{2} + \left(-\frac{36}{125}\right) \mathit{u0}_{0} \mathit{u3}_{3} \mathit{u3}_{0}^{2} + \left(-\frac{108}{125}\right) \mathit{u0}_{0} \mathit{u3}_{2} \mathit{u3}_{1} \mathit{u3}_{0} + \left(-\frac{36}{125}\right) \mathit{u1}_{3} \mathit{u2}_{0} \mathit{u3}_{0}^{2} + \frac{36}{125} \mathit{u1}_{3} \mathit{u3}_{1} \mathit{u3}_{0}^{2} + \left(-\frac{108}{125}\right) \mathit{u1}_{2} \mathit{u2}_{1} \mathit{u3}_{0}^{2} + \left(-\frac{36}{125}\right) \mathit{u1}_{2} \mathit{u2}_{0}^{2} \mathit{u3}_{0} + \left(-\frac{72}{125}\right) \mathit{u1}_{2} \mathit{u2}_{0} \mathit{u3}_{1} \mathit{u3}_{0} + \frac{108}{125} \mathit{u1}_{2} \mathit{u3}_{2} \mathit{u3}_{0}^{2} + \frac{108}{125} \mathit{u1}_{2} \mathit{u3}_{1}^{2} \mathit{u3}_{0} + \left(-\frac{36}{125}\right) \mathit{u1}_{1} \mathit{u1}_{0} \mathit{u2}_{0} \mathit{u3}_{0} + \frac{36}{125} \mathit{u1}_{1} \mathit{u1}_{0} \mathit{u3}_{1} \mathit{u3}_{0} + \left(-\frac{108}{125}\right) \mathit{u1}_{1} \mathit{u2}_{2} \mathit{u3}_{0}^{2} + \left(-\frac{72}{125}\right) \mathit{u1}_{1} \mathit{u2}_{1} \mathit{u2}_{0} \mathit{u3}_{0} + \left(-\frac{216}{125}\right) \mathit{u1}_{1} \mathit{u2}_{1} \mathit{u3}_{1} \mathit{u3}_{0} + \left(-\frac{72}{125}\right) \mathit{u1}_{1} \mathit{u2}_{0}^{2} \mathit{u3}_{1} + \left(-\frac{36}{125}\right) \mathit{u1}_{1} \mathit{u2}_{0} \mathit{u3}_{2} \mathit{u3}_{0} + \frac{72}{125} \mathit{u1}_{1} \mathit{u2}_{0} \mathit{u3}_{1}^{2} + \frac{108}{125} \mathit{u1}_{1} \mathit{u3}_{3} \mathit{u3}_{0}^{2} + \frac{324}{125} \mathit{u1}_{1} \mathit{u3}_{2} \mathit{u3}_{1} \mathit{u3}_{0} + \left(-\frac{36}{125}\right) \mathit{u1}_{0}^{2} \mathit{u2}_{1} \mathit{u3}_{0} + \left(-\frac{36}{125}\right) \mathit{u1}_{0}^

-------------------------------------------------------------------------------------------------------------------
Computed the almost commutator of order 10


P_10 = 1 \mathit{u0}_{3} \mathit{u3}_{0} z_{0} + 1 \mathit{u0}_{2} \mathit{u2}_{0} z_{0} + 3 \mathit{u0}_{2} \mathit{u3}_{0} z_{1} + 1 \mathit{u0}_{1} \mathit{u1}_{0} z_{0} + 2 \mathit{u0}_{1} \mathit{u2}_{0} z_{1} + 3 \mathit{u0}_{1} \mathit{u3}_{0} z_{2} + 1 \mathit{u0}_{0}^{2} z_{0} + 2 \mathit{u0}_{0} \mathit{u1}_{0} z_{1} + 2 \mathit{u0}_{0} \mathit{u2}_{0} z_{2} + 2 \mathit{u0}_{0} \mathit{u3}_{0} z_{3} + 1 \mathit{u1}_{3} \mathit{u3}_{0} z_{1} + 1 \mathit{u1}_{2} \mathit{u2}_{0} z_{1} + 3 \mathit{u1}_{2} \mathit{u3}_{0} z_{2} + 1 \mathit{u1}_{1} \mathit{u1}_{0} z_{1} + 2 \mathit{u1}_{1} \mathit{u2}_{0} z_{2} + 3 \mathit{u1}_{1} \mathit{u3}_{0} z_{3} + 1 \mathit{u1}_{0}^{2} z_{2} + 1 \mathit{u1}_{0} \mathit{u2}_{1} z_{2} + 2 \mathit{u1}_{0} \mathit{u2}_{0} z_{3} + 1 \mathit{u1}_{0} \mathit{u3}_{1} z_{3} + 2 \mathit{u1}_{0} \mathit{u3}_{0} z_{4} + 1 \mathit{u2}_{3} \mathit{u3}_{0} z_{2} + 1 \mathit{u2}_{2} \mathit{u2}_{0} z_{2} + 3 \mathit{u2}_{2} \mathit{u3}_{0} z_{3} + 2 \mathit{u2}_{1} \mathit{u2}_{0} z_{3} + 3 \mathit{u2}_{1} \mathit{u3}_{0} z_{4} + 1 \mathit{u2}_{0}^{2} z_{4} + 1 \mathit{u2}_{0} \mathit{u3}_{2} z_{3} + 2 \mathit{u2}_{0} \mathit{u3}_{1} z_{4} + 2 \mathit{u2}_{0} \mathit{u3}_{0} z_{5} + 1 \mathit{u3}_{3} \mathit{u3}_{0} z_{3} + 3 \mathit{u3}_{2} \mathit{u3}_{0} z_{4} + 3 \mathit{u3}_{1} \mathit{u3}_{0} z_{5} + 1 \mathit{u3}_{0}^{2} z_{6} + 1 \mathit{u0}_{5} z_{0} + 5 \mathit{u0}_{4} z_{1} + 10 \mathit{u0}_{3} z_{2} + 10 \mathit{u0}_{2} z_{3} + 5 \mathit{u0}_{1} z_{4} + 2 \mathit{u0}_{0} z_{5} + 1 \mathit{u1}_{5} z_{1} + 5 \mathit{u1}_{4} z_{2} + 10 \mathit{u1}_{3} z_{3} + 10 \mathit{u1}_{2} z_{4} + 5 \mathit{u1}_{1} z_{5} + 2 \mathit{u1}_{0} z_{6} + 1 \mathit{u2}_{5} z_{2} + 5 \mathit{u2}_{4} z_{3} + 10 \mathit{u2}_{3} z_{4} + 10 \mathit{u2}_{2} z_{5} + 5 \mathit{u2}_{1} z_{6} + 2 \mathit{u2}_{0} z_{7} + 1 \mathit{u3}_{5} z_{3} + 5 \mathit{u3}_{4} z_{4} + 10 \mathit{u3}_{3} z_{5} + 10 \mathit{u3}_{2} z_{6} + 5 \mathit{u3}_{1} z_{7} + 2 \mathit{u3}_{0} z_{8} + 1 z_{10}

... This commutes with L
CPU times: user 29.3 s, sys: 58.7 ms, total: 29.4 s
Wall time: 29.4 s


## Some experiments

In [4]:
%time P = quasi_commutting_generic_diff(5,11)[0]; z = P.parent().gens()[-1]; P

CPU times: user 37 s, sys: 38 ms, total: 37 s
Wall time: 37 s


99/3125*u2_0*u3_0^4*z_0 + (-99/3125)*u3_1*u3_0^4*z_0 + 99/15625*u3_0^5*z_1 + (-44/625)*u0_0*u3_0^3*z_0 + (-132/625)*u1_0*u2_0*u3_0^2*z_0 + 132/625*u1_0*u3_1*u3_0^2*z_0 + (-44/625)*u1_0*u3_0^3*z_1 + 66/625*u2_2*u3_0^3*z_0 + 66/625*u2_1*u2_0*u3_0^2*z_0 + 198/625*u2_1*u3_1*u3_0^2*z_0 + (-44/625)*u2_0^3*u3_0*z_0 + 132/625*u2_0^2*u3_1*u3_0*z_0 + (-66/625)*u2_0^2*u3_0^2*z_1 + 198/625*u2_0*u3_2*u3_0^2*z_0 + 176/625*u2_0*u3_1^2*u3_0*z_0 + 66/625*u2_0*u3_1*u3_0^2*z_1 + (-44/625)*u2_0*u3_0^3*z_2 + (-66/625)*u3_3*u3_0^3*z_0 + (-462/625)*u3_2*u3_1*u3_0^2*z_0 + 66/625*u3_2*u3_0^3*z_1 + (-264/625)*u3_1^3*u3_0*z_0 + 132/625*u3_1^2*u3_0^2*z_1 + (-11/625)*u3_0^4*z_3 + 33/125*u0_1*u2_0*u3_0*z_0 + (-11/25)*u0_1*u3_1*u3_0*z_0 + 33/125*u0_1*u3_0^2*z_1 + 66/125*u0_0*u1_0*u3_0*z_0 + (-33/125)*u0_0*u2_1*u3_0*z_0 + 33/125*u0_0*u2_0^2*z_0 + (-66/125)*u0_0*u2_0*u3_1*z_0 + 66/125*u0_0*u2_0*u3_0*z_1 + (-33/125)*u0_0*u3_2*u3_0*z_0 + (-11/125)*u0_0*u3_1^2*z_0 + (-33/125)*u0_0*u3_1*u3_0*z_1 + 33/125*u0_0*u3_0^2*z_2 + (-44/125)*u1_2*u2_0*u3_0*z_0 + 44/125*u1_2*u3_1*u3_0*z_0 + (-11/25)*u1_1*u2_1*u3_0*z_0 + (-66/125)*u1_1*u2_0*u3_1*z_0 + 33/125*u1_1*u2_0*u3_0*z_1 + 11/25*u1_1*u3_2*u3_0*z_0 + 66/125*u1_1*u3_1^2*z_0 + (-66/125)*u1_1*u3_1*u3_0*z_1 + 33/125*u1_1*u3_0^2*z_2 + 33/125*u1_0^2*u2_0*z_0 + (-33/125)*u1_0^2*u3_1*z_0 + 33/125*u1_0^2*u3_0*z_1 + (-11/25)*u1_0*u2_2*u3_0*z_0 + (-33/125)*u1_0*u2_1*u2_0*z_0 + (-11/25)*u1_0*u2_1*u3_1*z_0 + 33/125*u1_0*u2_0^2*z_1 + (-11/25)*u1_0*u2_0*u3_2*z_0 + (-33/125)*u1_0*u2_0*u3_1*z_1 + 66/125*u1_0*u2_0*u3_0*z_2 + 11/25*u1_0*u3_3*u3_0*z_0 + 143/125*u1_0*u3_2*u3_1*z_0 + (-11/25)*u1_0*u3_2*u3_0*z_1 + (-44/125)*u1_0*u3_1^2*z_1 + 33/125*u1_0*u3_0^2*z_3 + 121/625*u2_4*u3_0^2*z_0 + 22/125*u2_3*u2_0*u3_0*z_0 + 484/625*u2_3*u3_1*u3_0*z_0 + 11/25*u2_2*u2_1*u3_0*z_0 + (-33/125)*u2_2*u2_0^2*z_0 + 99/125*u2_2*u2_0*u3_1*z_0 + (-44/125)*u2_2*u2_0*u3_0*z_1 + 616/625*u2_2*u3_2*u3_0*z_0 + 374/625*u2_2*u3_1^2*z_0 + 22/125*u2_2*u3_1*u3_0*z_1 + (-33/125)*u2_1^2*u2_0*z_0 + 77/125*u2_1^2*u3_1*z_0 + (-33/125)*u2_1^2*u3_0*z_1 + 22/25*u2_1*u2_0*u3_2*z_0 + (-88/125)*u2_1*u2_0*u3_1*z_1 + 33/125*u2_1*u2_0*u3_0*z_2 + 429/625*u2_1*u3_3*u3_0*z_0 + 638/625*u2_1*u3_2*u3_1*z_0 + 33/125*u2_1*u3_2*u3_0*z_1 + 44/125*u2_1*u3_1^2*z_1 + (-77/125)*u2_1*u3_1*u3_0*z_2 + 33/125*u2_1*u3_0^2*z_3 + 11/125*u2_0^3*z_2 + 33/125*u2_0^2*u3_3*z_0 + (-33/125)*u2_0^2*u3_2*z_1 + 33/125*u2_0^2*u3_0*z_3 + 242/625*u2_0*u3_4*u3_0*z_0 + 77/125*u2_0*u3_3*u3_1*z_0 + 22/125*u2_0*u3_3*u3_0*z_1 + 11/25*u2_0*u3_2^2*z_0 + 77/125*u2_0*u3_2*u3_1*z_1 + (-44/125)*u2_0*u3_2*u3_0*z_2 + (-11/25)*u2_0*u3_1^2*z_2 + 33/125*u2_0*u3_1*u3_0*z_3 + 33/125*u2_0*u3_0^2*z_4 + (-121/625)*u3_5*u3_0^2*z_0 + (-836/625)*u3_4*u3_1*u3_0*z_0 + 121/625*u3_4*u3_0^2*z_1 + (-264/125)*u3_3*u3_2*u3_0*z_0 + (-1254/625)*u3_3*u3_1^2*z_0 + 594/625*u3_3*u3_1*u3_0*z_1 + (-1683/625)*u3_2^2*u3_1*z_0 + 363/625*u3_2^2*u3_0*z_1 + 132/125*u3_2*u3_1^2*z_1 + (-44/125)*u3_1^2*u3_0*z_3 + 33/125*u3_1*u3_0^2*z_4 + 11/125*u3_0^3*z_5 + 132/125*u0_4*u3_0*z_0 + 33/25*u0_3*u2_0*z_0 + 11/125*u0_3*u3_1*z_0 + 121/25*u0_3*u3_0*z_1 + 44/25*u0_2*u1_0*z_0 + 11/25*u0_2*u2_1*z_0 + 99/25*u0_2*u2_0*z_1 + (-11/125)*u0_2*u3_2*z_0 + 44/25*u0_2*u3_1*z_1 + 187/25*u0_2*u3_0*z_2 + 33/25*u0_1*u0_0*z_0 + 33/25*u0_1*u1_1*z_0 + 66/25*u0_1*u1_0*z_1 + 44/25*u0_1*u2_1*z_1 + 99/25*u0_1*u2_0*z_2 + (-44/125)*u0_1*u3_3*z_0 + 11/25*u0_1*u3_2*z_1 + 11/5*u0_1*u3_1*z_2 + 132/25*u0_1*u3_0*z_3 + 33/25*u0_0^2*z_1 + 22/25*u0_0*u1_2*z_0 + 33/25*u0_0*u1_1*z_1 + 66/25*u0_0*u1_0*z_2 + (-11/25)*u0_0*u2_3*z_0 + 22/25*u0_0*u2_2*z_1 + 33/25*u0_0*u2_1*z_2 + 66/25*u0_0*u2_0*z_3 + (-11/125)*u0_0*u3_4*z_0 + (-11/25)*u0_0*u3_3*z_1 + 22/25*u0_0*u3_2*z_2 + 33/25*u0_0*u3_1*z_3 + 66/25*u0_0*u3_0*z_4 + (-44/125)*u1_4*u2_0*z_0 + 44/125*u1_4*u3_1*z_0 + 132/125*u1_4*u3_0*z_1 + (-99/125)*u1_3*u2_1*z_0 + 33/25*u1_3*u2_0*z_1 + 99/125*u1_3*u3_2*z_0 + 121/25*u1_3*u3_0*z_2 + 44/25*u1_2*u1_0*z_1 + (-121/125)*u1_2*u2_2*z_0 + 22/25*u1_2*u2_1*z_1 + 99/25*u1_2*u2_0*z_2 + 121/125*u1_2*u3_3*z_0 + (-11/25)*u1_2*u3_2*z

In [7]:
show(P.parent())
w = P.parent().weight_func([5,4,3,2,0],[1])
for i in range(P.order(z)+1):
    print(f"{i}:{P.coefficient(z[i]).is_homogeneous(w)}, {w(P.coefficient(z[i]))}")

Ring of operator polynomials in (u0, u1, u2, u3, z) over Differential Ring [[Rational Field], (0,)]

0:True, 11
1:True, 10
2:True, 9
3:True, 8
4:True, 7
5:True, 6
6:True, 5
7:True, 4
8:True, 3
9:True, 2
10:True, 0
11:True, 0


In [4]:
%time P = quasi_commutting_generic_linear(5,11)[0]; z = P.parent().gens()[-1]; P

Number of ansatz constants: 293
[[], [], [], [], [], [], [], [], [], []]
Generator for the z's in Ring of operator polynomials in (u0, u1, u2, u3, z) over Differential Ring [[Rational Field], (0,)]
CPU times: user 19.1 s, sys: 109 ms, total: 19.2 s
Wall time: 19.2 s


z_11

In [5]:
show(P.parent())
w = P.parent().weight_func([5,4,3,2,0],[1])
for i in range(P.order(z)+1):
    print(f"{i}:{P.coefficient(z[i]).is_homogeneous(w)}, {w(P.coefficient(z[i]))}")

Ring of operator polynomials in (u0, u1, u2, u3, z) over Differential Ring [[Rational Field], (0,)]

0:True, 0
1:True, 0
2:True, 0
3:True, 0
4:True, 0
5:True, 0
6:True, 0
7:True, 0
8:True, 0
9:True, 0
10:True, 0
11:True, 0


# Example Grumbau

In [64]:
R.<Y0,Y1,Y2,Y3,Y4,c0,c1,c2,z> = DifferentialPolynomialRing(QQ)

In [65]:
R.gens()

(Y0_*, Y1_*, Y2_*, Y3_*, Y4_*, c0_*, c1_*, c2_*, z_*)

In [84]:
A = z[2] + c2[0]*(1/2)*z[0]
L4 = A(z=A) + 2*c1[0]*z[1] + (c1[1] + c0[0])*z[0]

In [89]:
L4.coefficient(z[0])

1/4*c2_0^2 + c0_0 + c1_1 + 1/2*c2_2

In [90]:
P6 = Y0[0]*z[0] + Y1[0]*z[1] + Y2[0]*z[2] + Y3[0]*z[3] + Y4[0]*z[4] + z[6]
P6

Y0_0*z_0 + Y1_0*z_1 + Y2_0*z_2 + Y3_0*z_3 + Y4_0*z_4 + z_6

In [93]:
C = L4(z=P6) - P6(z=L4)

In [97]:
S = DSystem([C.coefficient(z[i]) for i in range(3, C.order(z)+1)], variables=[Y0,Y1,Y2,Y3,Y4])

In [99]:
sol = S.solve_linear()

In [100]:
sol

{Y4_*: 3/2*c2_0,
 Y3_*: 3*c1_0 + 3*c2_1,
 Y2_*: 3/4*c2_0^2 + 3/2*c0_0 + 9/2*c1_1 + 7/2*c2_2,
 Y1_*: 3/2*c1_0*c2_0 + 3/2*c2_1*c2_0 + 3/2*c0_1 + 4*c1_2 + 2*c2_3,
 Y0_*: 1/8*c2_0^3 + 3/4*c0_0*c2_0 + 3/4*c1_1*c2_0 + 3/2*c1_0^2 + 3/4*c1_0*c2_1 + 3/4*c2_2*c2_0 + 1/2*c2_1^2 + 5/4*c0_2 + 5/4*c1_3 + 1/2*c2_4}

In [103]:
C_red = C(**{v.variable_name(): k for v,k in sol.items()})

In [120]:
C_red.coefficient(z[0])

3/2*c0_0*c1_0*c2_1 + 3*c1_2*c1_0*c2_0 + (-1/2)*c1_2*c2_1*c2_0 + 3*c1_1^2*c2_0 + 6*c1_1*c1_0^2 + 6*c1_1*c1_0*c2_1 + (-3/4)*c1_1*c2_1^2 + 3/2*c1_0^2*c2_2 + (-1/4)*c1_0*c2_2*c2_1 + 1/2*c0_4*c2_0 + (-1/2)*c0_3*c1_0 + 5/4*c0_3*c2_1 + (-3/2)*c0_2*c0_0 + (-9/2)*c0_2*c1_1 + c0_2*c2_2 + (-3/2)*c0_1^2 + (-11/2)*c0_1*c1_2 + 1/4*c0_1*c2_3 + (-3/2)*c0_0*c1_3 + 1/2*c1_5*c2_0 + 5/2*c1_4*c1_0 + 2*c1_4*c2_1 + 15/2*c1_3*c1_1 + 4*c1_3*c2_2 + 5*c1_2^2 + 7/2*c1_2*c2_3 + 3/2*c1_1*c2_4 + 1/4*c1_0*c2_5 + 1/4*c0_6 + 1/4*c1_7

In [124]:
C_red.coefficient(z[1])

6*c1_1*c1_0*c2_0 + 3*c1_0^2*c2_1 + c0_3*c2_0 + (-6)*c0_2*c1_0 + 3/2*c0_2*c2_1 + (-3)*c0_1*c0_0 + (-12)*c0_1*c1_1 + 1/2*c0_1*c2_2 + (-6)*c0_0*c1_2 + 2*c1_4*c2_0 + 5*c1_3*c1_0 + 7*c1_3*c2_1 + 10*c1_2*c1_1 + 9*c1_2*c2_2 + 5*c1_1*c2_3 + c1_0*c2_4 + 1/2*c0_5 + c1_6

In [110]:
C_red.coefficient(z[2])

(-6)*c0_1*c1_0 + (-6)*c0_0*c1_1 + 2*c1_3*c2_0 + 5*c1_2*c2_1 + 4*c1_1*c2_2 + c1_0*c2_3 + c1_5

In [128]:
A = C_red.coefficient(z[2])

In [129]:
A.inverse_operation(0)

(-6)*c0_0*c1_0 + 2*c1_2*c2_0 + 3*c1_1*c2_1 + c1_0*c2_2 + c1_4

In [130]:
B = C_red.coefficient(z[1])

In [133]:
B.inverse_operation(0) - A

3*c1_0^2*c2_0 + c0_2*c2_0 + 1/2*c0_1*c2_1 + (-3/2)*c0_0^2 + 5*c1_2*c1_0 + 5/2*c1_1^2 + 1/2*c0_4

In [134]:
E = C_red.coefficient(z[0])

In [142]:
small_E = E - (1/4)*A.derivative(times=2) - (1/2)*(B.inverse_operation(0) - A).derivative(times=2)

In [148]:
small_E +(1/4)*c2[1]*A.inverse_operation(0)

6*c1_1*c1_0^2 + c0_3*c1_0 - c0_1*c1_2